In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import cupy as cp
from numba import cuda
from numba import cuda, float32, int32
import math
import time
import matplotlib.pyplot as plt

/home/cs/grad/dasj/anaconda3/lib/python3.11/site-packages/cupy/_environment.py:540: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


In [2]:
import os
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/lib/cuda'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
'''
def filter_columns_by_threshold(cpu_df, gpu_df, threshold=1e-2):
    columns_to_drop = []

    for col in cpu_df.columns:
        if col in gpu_df.columns:
            difference = np.abs(cpu_df[col].values - gpu_df[col].values)
            
            if (difference > threshold).any():
                columns_to_drop.append(col)
    print(len(columns_to_drop))
    print(columns_to_drop)
    cpu_df = cpu_df.drop(columns=columns_to_drop)
    gpu_df = gpu_df.drop(columns=columns_to_drop)

    return cpu_df, gpu_df
'''

'\ndef filter_columns_by_threshold(cpu_df, gpu_df, threshold=1e-2):\n    columns_to_drop = []\n\n    for col in cpu_df.columns:\n        if col in gpu_df.columns:\n            difference = np.abs(cpu_df[col].values - gpu_df[col].values)\n            \n            if (difference > threshold).any():\n                columns_to_drop.append(col)\n    print(len(columns_to_drop))\n    print(columns_to_drop)\n    cpu_df = cpu_df.drop(columns=columns_to_drop)\n    gpu_df = gpu_df.drop(columns=columns_to_drop)\n\n    return cpu_df, gpu_df\n'

In [4]:
#filter_columns_by_threshold(cpu_results_df,gpu_results_df)

In [5]:
#cpu_results_df['PEP']

In [6]:
#gpu_results_df['PEP']

In [7]:
'''
def filter(cpu_df,gpu_df):
    columns=cpu_results_df[cpu_results_df!=gpu_results_df].dropna(axis=1).columns
    print(len(columns))
    print(columns)
    cpu_df=cpu_df.drop(columns=columns)
    gpu_df=gpu_df.drop(columns=columns)
    return cpu_df,gpu_df
'''

'\ndef filter(cpu_df,gpu_df):\n    columns=cpu_results_df[cpu_results_df!=gpu_results_df].dropna(axis=1).columns\n    print(len(columns))\n    print(columns)\n    cpu_df=cpu_df.drop(columns=columns)\n    gpu_df=gpu_df.drop(columns=columns)\n    return cpu_df,gpu_df\n'

In [8]:
#filter(cpu_results_df,gpu_results_df)

In [9]:
datasets=[]

return_datasets=[]
cpu_linear_correlations=[]
linear_correlations=[]
residual_calculations=[]
residual_linear_correlations=[]

dynamic_correlations=[]
cpu_dynamic_correlations=[]

In [10]:
cpu_returns=[]
gpu_returns=[]

## Change the year to access the files

In [12]:
year=2000

# Data Parsing function

In [13]:
def download_data(year):
    loc = os.path.join(os.getcwd(),'datasets','data'+str(year)+'.csv')
    print(loc)
    data = pd.read_csv(loc,index_col='Date')
    return data

In [14]:
data=download_data(year)

/data/cs/grad/dasj/COMP 7570/datasets/data2000.csv


In [15]:
data.head()

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,44.290768,72.063606,2.125906,0.609822,0.859423,6.228925,5.879150,10901.087891,0.102107,6.130104,...,890.625,6.885552,39.497936,2.744372,11.532703,4.861753,25.027779,5.652174,4.378227,39.500340
2000-01-04,40.907444,72.397484,2.183363,0.635232,0.786965,6.202757,5.810787,10122.438477,0.102107,6.148964,...,871.875,6.885552,37.660812,2.744372,11.453301,4.764031,24.666668,5.326087,4.378227,37.587597
2000-01-05,38.369949,76.571045,2.240821,0.640878,0.798481,6.228925,5.810787,10122.438477,0.102107,6.073520,...,871.875,7.004584,39.599995,2.724193,11.512856,4.788458,25.138889,5.108696,4.378227,37.543118
2000-01-06,36.908974,75.569412,2.125906,0.643701,0.729382,6.333615,5.810787,11679.737305,0.102107,6.092384,...,881.250,7.013742,38.783508,2.724193,11.433455,4.747740,23.777779,5.380435,4.264506,38.076908
2000-01-07,39.984726,75.346786,2.183363,0.621115,0.763932,6.542994,5.879150,11679.737305,0.102107,6.111243,...,843.750,6.995429,39.702057,2.724193,11.552554,4.641874,23.513889,5.326087,4.548805,38.165886


In [16]:
data.shape

(252, 1654)

In [17]:
data.isnull().sum().sum()

0

In [18]:
data=data.dropna()

In [19]:
data.shape

(252, 1654)

In [20]:
datasets.append(data)

# Returns Calculation

In [21]:
def calculate_returns_cpu(data):
    returns = np.zeros_like(data.values[1:], dtype=np.float64)  

    for col in range(data.shape[1]): 
        for row in range(1, data.shape[0]):  
            if data.iloc[row - 1, col] != 0:  
                returns[row - 1, col] = (data.iloc[row, col] - data.iloc[row - 1, col]) / data.iloc[row - 1, col]
    
    returns_df = pd.DataFrame(returns, columns=data.columns)  
    return returns_df

In [22]:
@cuda.jit
def calculate_returns_kernel(prices, returns):
    idx = cuda.grid(1) 
    
    if idx < prices.shape[1] and idx >= 0:  
        for i in range(1, prices.shape[0]):  
            if prices[i - 1, idx] != 0:  
                returns[i - 1, idx] = (prices[i, idx] - prices[i - 1, idx]) / prices[i - 1, idx]

In [23]:
def calculate_returns_gpu(data_values,columns):
    returns = np.zeros_like(data_values[1:], dtype=np.float64)

    closing_prices_device = cuda.to_device(np.array(data_values,dtype=np.float64))
    returns_device = cuda.to_device(np.array(returns,dtype=np.float64))

    device = cuda.get_current_device()
    max_threads_per_block = device.MAX_THREADS_PER_BLOCK
    threads_per_block = max(256, max_threads_per_block)
    blocks_per_grid = (data_values.shape[0] + (threads_per_block - 1)) // threads_per_block

    start_event = cuda.event()
    end_event = cuda.event()

    start_event.record()
    calculate_returns_kernel[blocks_per_grid, threads_per_block](closing_prices_device, returns_device)
    end_event.record()

    cuda.synchronize()
    elapsed_time = cuda.event_elapsed_time(start_event, end_event)

    returns_device.copy_to_host(returns)

    returns=pd.DataFrame(returns, columns=columns)
    return returns, elapsed_time

In [24]:
start_time = time.time()
returns_cpu = calculate_returns_cpu(data)
end_time = time.time()
cpu_elapsed_time = end_time - start_time
print(f"CPU execution time: {cpu_elapsed_time:.4f} seconds")


CPU execution time: 19.7097 seconds


In [25]:
import math
length=math.ceil(len(data.columns)/1024)
gpu_elapsed_time=0.0
print(length)
ret=[]
for i in range(1,length+1):
    d=data.iloc[:,(i-1)*1024:i*1024]
    returns_gpu, gpu_time = calculate_returns_gpu(d,d.columns)
    ret.append(returns_gpu)
    gpu_elapsed_time+=gpu_time

returns_gpu=pd.concat(ret,axis=1)
print(f"GPU execution time: {gpu_elapsed_time/1000:.4f} s")

2
GPU execution time: 0.1619 s


/home/cs/grad/dasj/anaconda3/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [26]:
gpu_results_df = pd.DataFrame(returns_gpu)
cpu_results_df = returns_cpu

In [27]:
gpu_results_df.shape

(251, 1654)

In [28]:
gpu_results_df.shape

(251, 1654)

In [29]:
(np.abs(cpu_results_df-gpu_results_df)>0).sum().sum()

0

In [30]:
if np.allclose(gpu_results_df.values, cpu_results_df.values, atol=1e-2):
    print("GPU and CPU results match!")
    return_datasets.append(gpu_results_df)
else:
    diff=np.abs(gpu_results_df.values-cpu_results_df.values)
    print(f"GPU and CPU results do not match. The max difference is : {np.max(diff)}")

GPU and CPU results match!


In [31]:
#cpu_results_df,gpu_results_df=filter_columns_by_threshold(cpu_results_df,gpu_results_df)
#cpu_results_df,gpu_results_df=filter(cpu_results_df,gpu_results_df)

In [32]:
if np.allclose(gpu_results_df.values, cpu_results_df.values, atol=1e-4):
    print("GPU and CPU results match!")
    return_datasets.append(gpu_results_df)
else:
    diff=np.abs(gpu_results_df.values-cpu_results_df.values)
    print(f"GPU and CPU results do not match. The max difference is : {np.max(diff)}")

GPU and CPU results match!


In [33]:
len(gpu_results_df.columns)

1654

In [34]:
(np.abs(gpu_results_df-cpu_results_df)>0).sum().sum()

0

In [35]:
#non_matching_gpu=gpu_results_df[cpu_results_df!=gpu_results_df].dropna(axis=1)
#non_matching_gpu

In [36]:
cpu_results_df

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
0,-0.076389,0.004633,0.027027,0.041667,-0.084310,-0.004201,-0.011628,-0.071429,0.000000,0.003077,...,-0.021053,0.000000,-0.046512,0.000000,-0.006885,-0.020100,-0.014428,-0.057692,0.000000,-0.048423
1,-0.062030,0.057648,0.026316,0.008889,0.014633,0.004219,0.000000,0.000000,0.000000,-0.012269,...,0.000000,0.017287,0.051491,-0.007353,0.005200,0.005127,0.019144,-0.040816,0.000000,-0.001183
2,-0.038076,-0.013081,-0.051282,0.004405,-0.086538,0.016807,0.000000,0.153846,0.000000,0.003106,...,0.010753,0.001307,-0.020618,0.000000,-0.006897,-0.008503,-0.054144,0.053191,-0.025974,0.014218
3,0.083333,-0.002946,0.027027,-0.035088,0.047369,0.033058,0.011765,0.000000,0.000000,0.003095,...,-0.042553,-0.002611,0.023684,0.000000,0.010417,-0.022298,-0.011098,-0.010101,0.066666,0.002337
4,0.060577,-0.002954,0.000000,0.000000,-0.017588,-0.018000,-0.023256,0.600000,0.000000,0.000000,...,0.044444,0.023560,-0.023136,0.000000,-0.006873,0.040351,0.033668,0.010204,0.025000,-0.018066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.064476,0.096456,0.066667,-0.006897,0.066667,0.002670,0.036232,0.051282,0.000000,0.044716,...,0.090226,0.039049,0.051948,-0.007195,-0.004238,-0.007767,0.025807,-0.142858,0.000000,0.060085
247,-0.021714,-0.007181,-0.093750,-0.020833,-0.020832,0.000000,0.006993,-0.048780,0.000000,-0.035019,...,0.048276,0.000170,-0.037037,-0.043478,-0.006383,-0.007828,-0.026730,-0.250001,0.014706,-0.004048
248,0.045561,-0.010850,0.000000,0.014185,0.008510,0.038615,0.041666,0.012821,0.074626,-0.002017,...,0.003289,0.060557,0.025641,0.046153,-0.008565,0.019724,0.033926,0.222223,0.000000,0.002032
249,-0.011174,0.012797,0.000000,-0.013986,0.000000,0.021795,-0.006756,-0.012658,0.000000,0.002021,...,-0.022951,0.052469,0.000000,-0.029412,0.084233,0.034816,0.098437,-0.181819,-0.043479,0.007100


In [37]:
cpu_returns.append(cpu_results_df)

In [38]:
gpu_results_df

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
0,-0.076389,0.004633,0.027027,0.041667,-0.084310,-0.004201,-0.011628,-0.071429,0.000000,0.003077,...,-0.021053,0.000000,-0.046512,0.000000,-0.006885,-0.020100,-0.014428,-0.057692,0.000000,-0.048423
1,-0.062030,0.057648,0.026316,0.008889,0.014633,0.004219,0.000000,0.000000,0.000000,-0.012269,...,0.000000,0.017287,0.051491,-0.007353,0.005200,0.005127,0.019144,-0.040816,0.000000,-0.001183
2,-0.038076,-0.013081,-0.051282,0.004405,-0.086538,0.016807,0.000000,0.153846,0.000000,0.003106,...,0.010753,0.001307,-0.020618,0.000000,-0.006897,-0.008503,-0.054144,0.053191,-0.025974,0.014218
3,0.083333,-0.002946,0.027027,-0.035088,0.047369,0.033058,0.011765,0.000000,0.000000,0.003095,...,-0.042553,-0.002611,0.023684,0.000000,0.010417,-0.022298,-0.011098,-0.010101,0.066666,0.002337
4,0.060577,-0.002954,0.000000,0.000000,-0.017588,-0.018000,-0.023256,0.600000,0.000000,0.000000,...,0.044444,0.023560,-0.023136,0.000000,-0.006873,0.040351,0.033668,0.010204,0.025000,-0.018066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.064476,0.096456,0.066667,-0.006897,0.066667,0.002670,0.036232,0.051282,0.000000,0.044716,...,0.090226,0.039049,0.051948,-0.007195,-0.004238,-0.007767,0.025807,-0.142858,0.000000,0.060085
247,-0.021714,-0.007181,-0.093750,-0.020833,-0.020832,0.000000,0.006993,-0.048780,0.000000,-0.035019,...,0.048276,0.000170,-0.037037,-0.043478,-0.006383,-0.007828,-0.026730,-0.250001,0.014706,-0.004048
248,0.045561,-0.010850,0.000000,0.014185,0.008510,0.038615,0.041666,0.012821,0.074626,-0.002017,...,0.003289,0.060557,0.025641,0.046153,-0.008565,0.019724,0.033926,0.222223,0.000000,0.002032
249,-0.011174,0.012797,0.000000,-0.013986,0.000000,0.021795,-0.006756,-0.012658,0.000000,0.002021,...,-0.022951,0.052469,0.000000,-0.029412,0.084233,0.034816,0.098437,-0.181819,-0.043479,0.007100


In [39]:
cpu_results_df=cpu_results_df.dropna()
gpu_results_df=gpu_results_df.dropna()

In [40]:
gpu_returns.append(gpu_results_df)
cpu_returns.append(cpu_results_df)

In [41]:
print(cpu_returns[-1].shape)
print(gpu_returns[-1].shape)

(251, 1654)
(251, 1654)


# Linear Correlation

In [42]:
def cpu_correlation(data):
    import time
    
    data = np.asarray(data, dtype=np.float32)
    
    means = np.mean(data, axis=0)
    
    std_devs = np.std(data, axis=0)
    
    n_cols = data.shape[1]
    correlations = np.zeros((n_cols, n_cols), dtype=np.float32)
    start=time.time()
    for i in range(n_cols):
        for j in range(n_cols):
            if i == j:
                correlations[i, j] = 1.0  
            else:
                cov = np.sum((data[:, i] - means[i]) * (data[:, j] - means[j]))
                correlations[i, j] = cov / (std_devs[i] * std_devs[j] * len(data))
    end=time.time()
    
    return correlations, end-start

In [43]:
@cuda.jit
def calculate_correlation(data, means, std_devs, correlations):
    col_idx1, col_idx2 = cuda.grid(2)  

    n_rows = data.shape[0]
    n_cols = data.shape[1]

    if col_idx1 < n_cols and col_idx2 < n_cols:
        cov = 0.0  
        var1 = 0.0  
        var2 = 0.0  

        for i in range(n_rows):
            val1 = data[i, col_idx1] - means[col_idx1]
            val2 = data[i, col_idx2] - means[col_idx2]

            cov += val1 * val2
            var1 += val1 * val1
            var2 += val2 * val2

        if var1 > 0 and var2 > 0:
            correlations[col_idx1, col_idx2] = cov / (math.sqrt(var1) * math.sqrt(var2))
        else:
            correlations[col_idx1, col_idx2] = 0.0

In [44]:
def gpu_correlation(data):
    data = np.asarray(data, dtype=np.float32)

    data_device = cuda.to_device(data)
    means_device = cuda.to_device(np.mean(data, axis=0).astype(np.float32))
    std_devs_device = cuda.to_device(np.std(data, axis=0).astype(np.float32))
    correlations_device = cuda.device_array((data.shape[1], data.shape[1]), dtype=np.float32)

    device = cuda.get_current_device()
    max_threads_per_block = device.MAX_THREADS_PER_BLOCK
    side_length = int(np.sqrt(max_threads_per_block)) 
    threads_per_block = (side_length, side_length) 
    
    blocks_per_grid_x = (data.shape[1] + threads_per_block[0] - 1) // threads_per_block[0]
    blocks_per_grid_y = (data.shape[1] + threads_per_block[1] - 1) // threads_per_block[1]
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    start_event = cuda.event()
    end_event = cuda.event()

    start_event.record()

    calculate_correlation[blocks_per_grid, threads_per_block](
        data_device, means_device, std_devs_device, correlations_device
    )

    end_event.record()
    cuda.synchronize()
    
    elapsed_time = cuda.event_elapsed_time(start_event, end_event)

    
    correlations_result = correlations_device.copy_to_host()

    return correlations_result, elapsed_time

In [45]:
gpu_corr,gpu_time=gpu_correlation(gpu_results_df)

In [46]:
print("Elapsed time on GPU is : ",gpu_time/1000)

Elapsed time on GPU is :  0.17631539916992187


In [47]:
cpu_corr,cpu_time=cpu_correlation(cpu_results_df)

In [48]:
print("Elapsed time on CPU is : ",cpu_time)

Elapsed time on CPU is :  18.958308219909668


In [49]:
start_time=time.time()
bultin_corr=cpu_results_df.corr()
print('Elapsed time in built-in corr function is: ',time.time()-start_time)

Elapsed time in built-in corr function is:  1.0780754089355469


In [50]:
gpu_corr_df=pd.DataFrame(gpu_corr,columns=gpu_results_df.columns,index=gpu_results_df.columns)
cpu_corr_df=pd.DataFrame(cpu_corr,index=cpu_results_df.columns,columns=cpu_results_df.columns)

In [51]:
diff = np.abs(cpu_corr_df - gpu_corr_df)
threshold = 1e-5

if np.all(diff < threshold):
    print("The results from CPU and GPU are consistent within the threshold.")
    linear_correlations.append(pd.DataFrame(gpu_corr_df, columns=gpu_results_df.columns, index=gpu_results_df.index))
else:
    row_indices, col_indices = np.where(diff >= threshold)
    
    differing_rows = cpu_corr_df.index[row_indices]
    differing_cols = cpu_corr_df.columns[col_indices]
    
    print(f"The results differ beyond the threshold. Max difference: {np.max(diff)}")
    print("Differing rows and columns:")
    for row, col in zip(differing_rows, differing_cols):
        print(f"Row: {row}, Column: {col}")


The results from CPU and GPU are consistent within the threshold.


In [52]:
if len(differing_rows)!=0 and len(differing_cols)!=0 :
    cpu_corr_df=cpu_corr_df.drop(index=differing_rows,columns=differing_cols)
    gpu_corr_df=gpu_corr_df.drop(index=differing_rows,columns=differing_cols)

NameError: name 'differing_rows' is not defined

In [ ]:
if len(differing_cols)!=0 :
    cpu_returns[-1]=cpu_returns[-1].drop(columns=differing_cols)
    gpu_returns[-1]=gpu_returns[-1].drop(columns=differing_cols)

In [53]:
print(cpu_corr_df.shape)
print(gpu_corr_df.shape)
print(gpu_returns[-1].shape)
print(cpu_returns[-1].shape)

(1654, 1654)
(1654, 1654)
(251, 1654)
(251, 1654)


In [54]:
diff = np.abs(cpu_corr_df - gpu_corr_df)

if np.all(diff < 1e-5):
    print("The results from CPU and GPU are consistent within the threshold.")
    linear_correlations.append(gpu_corr_df)
    cpu_linear_correlations.append(cpu_corr_df)
else:
    print(f"The results differ beyond the threshold. Max difference: {np.max(diff)}")

The results from CPU and GPU are consistent within the threshold.


In [55]:
print(linear_correlations[-1].shape)
print(cpu_linear_correlations[-1].shape)

(1654, 1654)
(1654, 1654)


In [56]:
gpu_corr_df

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
A,1.000000,0.060055,0.105745,0.068161,0.187420,0.163134,0.112485,0.149201,-0.020993,0.178850,...,0.265733,0.024445,0.137632,-0.016805,0.063693,0.161885,0.039665,0.078390,0.151110,0.137440
AA,0.060055,1.000000,0.040147,-0.052112,-0.073040,0.178934,0.162735,-0.059906,0.033063,0.038447,...,0.132752,0.156911,0.092623,-0.032667,0.167220,0.197395,-0.023940,0.019908,-0.065141,0.189320
AAME,0.105745,0.040147,1.000000,-0.027005,-0.003294,0.063666,0.025925,-0.023721,0.085917,0.147374,...,0.007508,-0.081942,0.000194,0.086593,0.075938,-0.006023,-0.009842,0.071114,-0.051990,-0.053066
AAON,0.068161,-0.052112,-0.027005,1.000000,0.019104,0.036082,-0.003111,0.046418,-0.006045,0.011403,...,0.024203,0.162109,-0.024780,0.004654,-0.045923,-0.080574,0.106002,0.035837,-0.066699,-0.033045
AAPL,0.187420,-0.073040,-0.003294,0.019104,1.000000,0.121327,-0.046102,0.011134,-0.026474,0.091759,...,0.185977,0.015197,0.172271,0.089980,-0.056203,0.031793,0.239782,0.010591,0.030175,0.122205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.161885,0.197395,-0.006023,-0.080574,0.031793,0.260947,0.053375,0.080772,0.044335,-0.004757,...,-0.009276,0.096250,0.158490,0.038880,0.082647,1.000000,0.075392,-0.041410,0.138217,0.225962
ZBRA,0.039665,-0.023940,-0.009842,0.106002,0.239782,0.159299,0.086417,0.091207,0.033217,0.192230,...,0.038478,0.068191,0.113366,0.038284,0.006531,0.075392,1.000000,-0.080805,0.037998,0.100075
ZD,0.078390,0.019908,0.071114,0.035837,0.010591,0.012549,0.102757,0.069893,0.188665,0.081197,...,0.076257,0.045094,-0.016395,0.115815,0.023252,-0.041410,-0.080805,1.000000,0.039170,-0.039563
ZEUS,0.151110,-0.065141,-0.051990,-0.066699,0.030175,0.024711,-0.009316,-0.018788,0.099016,0.037436,...,-0.037441,0.054530,0.124562,-0.052521,0.003075,0.138217,0.037998,0.039170,1.000000,0.086746


In [57]:
cpu_corr_df

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
A,1.000000,0.060055,0.105745,0.068161,0.187420,0.163134,0.112485,0.149201,-0.020993,0.178850,...,0.265733,0.024445,0.137632,-0.016805,0.063693,0.161885,0.039665,0.078390,0.151110,0.137440
AA,0.060055,1.000000,0.040147,-0.052112,-0.073040,0.178934,0.162735,-0.059906,0.033063,0.038447,...,0.132752,0.156911,0.092623,-0.032667,0.167220,0.197395,-0.023940,0.019908,-0.065141,0.189320
AAME,0.105745,0.040147,1.000000,-0.027005,-0.003294,0.063666,0.025925,-0.023721,0.085917,0.147374,...,0.007508,-0.081942,0.000194,0.086593,0.075938,-0.006023,-0.009842,0.071114,-0.051990,-0.053066
AAON,0.068161,-0.052112,-0.027005,1.000000,0.019104,0.036082,-0.003111,0.046418,-0.006045,0.011403,...,0.024203,0.162109,-0.024780,0.004654,-0.045923,-0.080574,0.106002,0.035837,-0.066699,-0.033045
AAPL,0.187420,-0.073040,-0.003294,0.019104,1.000000,0.121327,-0.046102,0.011134,-0.026474,0.091759,...,0.185977,0.015197,0.172271,0.089980,-0.056203,0.031793,0.239782,0.010591,0.030175,0.122204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.161885,0.197395,-0.006023,-0.080574,0.031793,0.260947,0.053375,0.080772,0.044335,-0.004758,...,-0.009276,0.096250,0.158490,0.038880,0.082647,1.000000,0.075392,-0.041410,0.138217,0.225962
ZBRA,0.039665,-0.023940,-0.009842,0.106002,0.239782,0.159299,0.086417,0.091207,0.033217,0.192230,...,0.038478,0.068191,0.113366,0.038284,0.006531,0.075392,1.000000,-0.080805,0.037998,0.100075
ZD,0.078390,0.019908,0.071114,0.035837,0.010591,0.012549,0.102757,0.069893,0.188665,0.081197,...,0.076257,0.045094,-0.016395,0.115815,0.023252,-0.041410,-0.080805,1.000000,0.039170,-0.039563
ZEUS,0.151110,-0.065141,-0.051990,-0.066699,0.030175,0.024711,-0.009316,-0.018788,0.099016,0.037436,...,-0.037441,0.054530,0.124562,-0.052521,0.003075,0.138217,0.037998,0.039170,1.000000,0.086746


In [58]:
gpu_corr_df

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
A,1.000000,0.060055,0.105745,0.068161,0.187420,0.163134,0.112485,0.149201,-0.020993,0.178850,...,0.265733,0.024445,0.137632,-0.016805,0.063693,0.161885,0.039665,0.078390,0.151110,0.137440
AA,0.060055,1.000000,0.040147,-0.052112,-0.073040,0.178934,0.162735,-0.059906,0.033063,0.038447,...,0.132752,0.156911,0.092623,-0.032667,0.167220,0.197395,-0.023940,0.019908,-0.065141,0.189320
AAME,0.105745,0.040147,1.000000,-0.027005,-0.003294,0.063666,0.025925,-0.023721,0.085917,0.147374,...,0.007508,-0.081942,0.000194,0.086593,0.075938,-0.006023,-0.009842,0.071114,-0.051990,-0.053066
AAON,0.068161,-0.052112,-0.027005,1.000000,0.019104,0.036082,-0.003111,0.046418,-0.006045,0.011403,...,0.024203,0.162109,-0.024780,0.004654,-0.045923,-0.080574,0.106002,0.035837,-0.066699,-0.033045
AAPL,0.187420,-0.073040,-0.003294,0.019104,1.000000,0.121327,-0.046102,0.011134,-0.026474,0.091759,...,0.185977,0.015197,0.172271,0.089980,-0.056203,0.031793,0.239782,0.010591,0.030175,0.122205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.161885,0.197395,-0.006023,-0.080574,0.031793,0.260947,0.053375,0.080772,0.044335,-0.004757,...,-0.009276,0.096250,0.158490,0.038880,0.082647,1.000000,0.075392,-0.041410,0.138217,0.225962
ZBRA,0.039665,-0.023940,-0.009842,0.106002,0.239782,0.159299,0.086417,0.091207,0.033217,0.192230,...,0.038478,0.068191,0.113366,0.038284,0.006531,0.075392,1.000000,-0.080805,0.037998,0.100075
ZD,0.078390,0.019908,0.071114,0.035837,0.010591,0.012549,0.102757,0.069893,0.188665,0.081197,...,0.076257,0.045094,-0.016395,0.115815,0.023252,-0.041410,-0.080805,1.000000,0.039170,-0.039563
ZEUS,0.151110,-0.065141,-0.051990,-0.066699,0.030175,0.024711,-0.009316,-0.018788,0.099016,0.037436,...,-0.037441,0.054530,0.124562,-0.052521,0.003075,0.138217,0.037998,0.039170,1.000000,0.086746


In [59]:
len(gpu_corr_df.columns)

1654

In [60]:
len(cpu_corr_df.columns)

1654

In [61]:
len(data.dropna().pct_change().dropna().corr().columns)

1654

In [62]:
cpu_corr_df=cpu_corr_df.dropna()
gpu_corr_df=gpu_corr_df.dropna()

In [63]:
gpu_corr_df.isnull().sum().sum()

0

# Finding Optimal Alpha

In [64]:
def choose_optimal_alpha_cpu(Z):
    Z = np.array(Z, dtype=np.float64)  
    alpha_values = np.arange(0.01, 0.3, 0.01, dtype=np.float32)  
    t = len(Z)
    cut_t = math.ceil(t * (2 / 3))
    k = math.ceil(t * (1 / 3))
    
    mse_alpha = np.zeros(len(alpha_values), dtype=np.float32)  
    s=0
    c=0
    for i in range(k+1):
        s+=Z[i]
        c+=1

    s/=c
    p=s.copy()
    for a, alpha in enumerate(alpha_values):
        error = 0  
        c=0
        t=0
        for i in range(1, cut_t):
            e = Z[i] - s
            if alpha==len(alpha_values)-1:
                print(f'Z[{i}]: {Z[i]}\n')
            if i>k:
                error+=e**2
                c+=1
            s = alpha * Z[i] + (1 - alpha) * s
            t=Z[i]
        mse_alpha[a]=error/c
          
    alpha_opt = alpha_values[np.argmin(mse_alpha)]  
    return alpha_opt

In [65]:
@cuda.jit
def mse_compute(Z, alphas, mse, cut_t, k,s):
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

    if idx < alphas.size and idx>=0:    
        alpha = alphas[idx]
        error = 0.0
        count=0.0
        p=s
        t=0.0
        for i in range(1, cut_t):
            e = Z[i] - p
            
            if i > k:
                error += e ** 2
                count+=1.0
            p = alpha * Z[i] + (1 - alpha) * p
            t=Z[i]
        
        mse[idx] = error / count
        
    cuda.syncthreads()
    
def gpu_alpha(Z):
    Z = np.array(Z, dtype=np.float64)
    alpha_values = np.arange(0.01, 0.3, 0.01, dtype=np.float64)
    t = len(Z)
    cut_t = math.ceil(t * (2 / 3))
    k = math.ceil(t * (1 / 3))
    n_alpha = len(alpha_values)
    s = np.mean(Z[:k+1])
    
    Z_device = cuda.to_device(Z)
    alpha_values_device = cuda.to_device(alpha_values)
    mse_alpha_device = cuda.device_array(n_alpha, dtype=np.float64)

    device = cuda.get_current_device()
    max_threads_per_block = device.MAX_THREADS_PER_BLOCK
    threads_per_block = min(256, max_threads_per_block)
    blocks_per_grid = math.ceil(n_alpha / threads_per_block)

    start_event = cuda.event()
    end_event = cuda.event()
    start_event.record()

    mse_compute[blocks_per_grid, threads_per_block](
        Z_device, alpha_values_device, mse_alpha_device, cut_t, k,s
    )

    end_event.record()
    cuda.synchronize()

    elapsed_time_ms = cuda.event_elapsed_time(start_event, end_event)

    mse_alpha = mse_alpha_device.copy_to_host()

    alpha_opt = alpha_values[np.argmin(mse_alpha)]

    return alpha_opt, elapsed_time_ms

In [66]:
y=np.array(cpu_results_df.AAPL)
y_mean=np.mean(y)
rho = np.corrcoef(y - y_mean, np.sign(y - y_mean))[0, 1]
vol = np.abs(y - y_mean) / rho

In [67]:
alpha_gpu,_=gpu_alpha(vol)
alpha_cpu=choose_optimal_alpha_cpu(vol)

/home/cs/grad/dasj/anaconda3/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [68]:
alpha_gpu

0.04

In [69]:
alpha_cpu

0.03

In [70]:
'''
@cuda.jit
def compute_mse(Z, alpha_values, mse_alpha, k, cut_t):
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    n_alpha = len(alpha_values)
    
    if idx < n_alpha:
        alpha = alpha_values[idx]
        
        # Initialize `s` using the same logic as the CPU version
        s = 0.0
        for j in range(k + 1):
            s += Z[j]
        s /= (k + 1)
        
        mse = 0.0
        
        # Calculate MSE
        for i in range(k + 1, cut_t):
            error = Z[i] - s
            s = alpha * Z[i] + (1 - alpha) * s
            mse += error ** 2
        
        # Normalize MSE to mean squared error
        mse /= (cut_t - k - 1)
        
        # Store the result in the corresponding alpha index
        mse_alpha[idx] = mse

def gpu_alpha(Z):
    """
    GPU implementation to find the optimal alpha value that minimizes MSE.
    Returns the optimal alpha and elapsed time in milliseconds.
    """
    Z = np.array(Z, dtype=np.float64)  # Ensure input array is float64
    alpha_values = np.arange(0.01, 0.3, 0.01, dtype=np.float64)  # Alpha values as float64
    t = len(Z)
    cut_t = math.ceil(t * (2 / 3))
    k = math.ceil(t * (1 / 3))

    # Number of alpha values
    n_alpha = len(alpha_values)
    
    # Allocate device memory
    Z_device = cuda.to_device(Z)  # Device array of float64
    alpha_values_device = cuda.to_device(alpha_values)  # Device array of float64
    mse_alpha_device = cuda.device_array(n_alpha, dtype=np.float64)  # Device array to store MSE results

    # Dynamic grid and block size allocation
    device = cuda.get_current_device()
    max_threads_per_block = device.MAX_THREADS_PER_BLOCK
    threads_per_block = min(256, max_threads_per_block)  # Safe upper limit for threads per block
    blocks_per_grid = math.ceil(n_alpha / threads_per_block)

    # Record start time
    start_event = cuda.event()
    end_event = cuda.event()
    start_event.record()

    # Launch kernel
    compute_mse[blocks_per_grid, threads_per_block](
        Z_device, alpha_values_device, mse_alpha_device, k, cut_t
    )

    # Record end time and synchronize
    end_event.record()
    cuda.synchronize()

    # Measure elapsed time in milliseconds
    elapsed_time_ms = cuda.event_elapsed_time(start_event, end_event)

    # Copy MSE values back to host
    mse_alpha = mse_alpha_device.copy_to_host()

    # Find the optimal alpha value (minimizing MSE)
    alpha_opt = alpha_values[np.argmin(mse_alpha)]
    
    return alpha_opt, elapsed_time_ms
'''



'\n@cuda.jit\ndef compute_mse(Z, alpha_values, mse_alpha, k, cut_t):\n    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x\n    n_alpha = len(alpha_values)\n    \n    if idx < n_alpha:\n        alpha = alpha_values[idx]\n        \n        # Initialize `s` using the same logic as the CPU version\n        s = 0.0\n        for j in range(k + 1):\n            s += Z[j]\n        s /= (k + 1)\n        \n        mse = 0.0\n        \n        # Calculate MSE\n        for i in range(k + 1, cut_t):\n            error = Z[i] - s\n            s = alpha * Z[i] + (1 - alpha) * s\n            mse += error ** 2\n        \n        # Normalize MSE to mean squared error\n        mse /= (cut_t - k - 1)\n        \n        # Store the result in the corresponding alpha index\n        mse_alpha[idx] = mse\n\ndef gpu_alpha(Z):\n    """\n    GPU implementation to find the optimal alpha value that minimizes MSE.\n    Returns the optimal alpha and elapsed time in milliseconds.\n    """\n    Z = np.array(

In [71]:
optimal_alpha, time_taken = gpu_alpha(gpu_results_df.AAPL)

print(f"Optimal alpha: {optimal_alpha}")
print(f"Time taken: {time_taken/1000} s")

Optimal alpha: 0.01
Time taken: 0.0005642240047454834 s


In [72]:
optimal_alpha, time_taken = gpu_alpha(gpu_results_df.AAPL)

print(f"Optimal alpha: {optimal_alpha}")
print(f"Time taken: {time_taken/1000} s")

Optimal alpha: 0.01
Time taken: 0.000596992015838623 s


In [73]:
import time
st=time.time()
print("Optimal Alpha by CPU code: ",choose_optimal_alpha_cpu(cpu_results_df.AAPL))
print('CPU_time: ', time.time()-st)           

Optimal Alpha by CPU code:  0.01
CPU_time:  0.052109479904174805


In [74]:
y=np.array(cpu_results_df.AAPL)
y_mean=np.mean(y)
rho = np.corrcoef(y - y_mean, np.sign(y - y_mean))[0, 1]
vol = np.abs(y - y_mean) / rho

In [75]:
y=np.array(gpu_results_df.AAPL)
y_mean=np.mean(y)
rho = np.corrcoef(y - y_mean, np.sign(y - y_mean))[0, 1]
vol2 = np.abs(y - y_mean) / rho

In [76]:
choose_optimal_alpha_cpu(vol)

0.03

In [77]:
gpu_alpha(vol)

(0.04, 0.3962880074977875)

In [78]:
start_time_cpu = time.time()

y=np.array(cpu_results_df.AAPL)
y_mean=np.mean(y)
rho = np.corrcoef(y - y_mean, np.sign(y - y_mean))[0, 1]
vol = np.abs(y - y_mean) / rho

cpu_results = choose_optimal_alpha_cpu(vol)
cpu_time = time.time() - start_time_cpu

gpu_results, gpu_time = gpu_alpha(vol)

is_valid = np.allclose(cpu_results, gpu_results, atol=1e-2)

print(is_valid) 
if is_valid:
    print("GPU and CPU results match")
else:
    print("GPU and CPU results don't match")
print({
    "cpu_results": cpu_results,
    "gpu_results": gpu_results,
    "cpu_time_seconds": cpu_time,
    "gpu_time_seconds": gpu_time / 1000,  # Convert ms to seconds
    "comparison_valid": is_valid,
})

True
GPU and CPU results match
{'cpu_results': 0.03, 'gpu_results': 0.04, 'cpu_time_seconds': 0.045845985412597656, 'gpu_time_seconds': 0.000159743994474411, 'comparison_valid': True}


# Volatility Calculation Function

In [79]:
def dd_vol_cpu(y):
    y = np.array(y, dtype=np.float32)  # Use float32 for consistency
    y_mean = np.mean(y)

    if np.isnan(y_mean) or np.isnan(np.std(y)):
        raise ValueError("Input data contains NaN values.")
        
    rho = np.corrcoef(y - y_mean, np.sign(y - y_mean))[0, 1]
    
    vol = np.abs(y - y_mean) / rho
    
    alpha = choose_optimal_alpha_cpu(vol)
    
    t = len(y)
    k = math.ceil(t * (1/3))
    s = np.mean(vol[:k+1], dtype=np.float32)  # Explicit float32

    s_t = []
    for i in range(1, t):
        s = alpha * vol[i] + (1 - alpha) * s
        s_t.append(s)
    
    return s_t[-1]  # Match GPU output


In [80]:
@cuda.jit
def dd_vol_gpu(vol,s_0, alpha, s, s_t, t):
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

    if idx == 0:
        s[0] = s_0

    cuda.syncthreads()

    if idx < t:
        temp_s = s[0]
        alpha_f32 = float32(alpha[0])  # Cast alpha to float32
        for i in range(idx, t):
            temp_s = alpha_f32 * vol[i] + (1 - alpha_f32) * temp_s
            s_t[i] = temp_s

In [81]:
def gpu_dd_vol(y):
    y=np.array(y, dtype=np.float64)
    y_mean = np.mean(y)

    if np.isnan(y_mean) or np.isnan(np.std(y)):
        raise ValueError("Input data contains NaN values.")
    
    rho = np.corrcoef(y - y_mean, np.sign(y - y_mean))[0, 1]
    vol = np.abs(y - y_mean) / rho

    alpha, _ = gpu_alpha(vol) 
    
    t = len(vol)
    
    k=math.ceil(t*(1/3))
    s_0 = np.mean(vol[:k+1])
    
    vol_device = cuda.to_device(vol)
    s_t_device = cuda.device_array(t, dtype=np.float64)
    s_device = cuda.device_array(1, dtype=np.float64)  
    alpha_device = cuda.device_array(1, dtype=np.float64)
    alpha_device[0] = alpha 

    device = cuda.get_current_device()
    max_threads_per_block = device.MAX_THREADS_PER_BLOCK
    threads_per_block = min(256, max_threads_per_block)  
    blocks_per_grid = math.ceil(t / threads_per_block)

    start_event = cuda.event()
    end_event = cuda.event()

    start_event.record()

    dd_vol_gpu[blocks_per_grid, threads_per_block](vol_device,s_0, alpha_device, s_device, s_t_device, t)

    end_event.record()

    cuda.synchronize()

    elapsed_time_ms = cuda.event_elapsed_time(start_event, end_event)

    s_t = s_t_device.copy_to_host()

    return s_t[-1], elapsed_time_ms 

In [82]:
import time
start=time.time()
print(dd_vol_cpu(cpu_results_df.AAPL))
print("Elapsed time on CPU: ",(time.time()-start), 'sec')

0.04857904348803542
Elapsed time on CPU:  0.052008628845214844 sec


In [83]:
value,time=gpu_dd_vol(gpu_results_df.AAPL)
print(value);
print(time/1000)

0.04582574430164658
0.05074534225463867


/home/cs/grad/dasj/anaconda3/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [84]:
import time
start_time_cpu = time.time()
cpu_results = dd_vol_cpu(cpu_results_df.AAPL)
cpu_time = time.time() - start_time_cpu

gpu_results, gpu_time = gpu_dd_vol(gpu_results_df.AAPL)

is_valid = np.allclose(cpu_results, gpu_results, atol=1e-2)

print(is_valid)

if is_valid:
    print('GPU and CPU results match')
else:
    print('GPU and CPU results don\'t match')
print({
    "cpu_results": cpu_results,
    "gpu_results": gpu_results,
    "cpu_time_seconds": cpu_time,
    "gpu_time_seconds": gpu_time / 1000,  # Convert ms to seconds
    "comparison_valid": is_valid,
})

True
GPU and CPU results match
{'cpu_results': 0.04857904348803542, 'gpu_results': 0.04582574430164658, 'cpu_time_seconds': 0.014929771423339844, 'gpu_time_seconds': 0.00012083200365304947, 'comparison_valid': True}


/home/cs/grad/dasj/anaconda3/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


# Residual Calculations

In [85]:
def dd_res_cpu(y):
    y = np.array(y, dtype=np.float64)
    
    y_mean = np.mean(y)
    
    rho = np.corrcoef(y - y_mean, np.sign(y - y_mean))[0, 1]
    
    vol = np.abs(y - y_mean) / rho
    
    alpha = choose_optimal_alpha_cpu(vol)
    
    t = len(y)
    cut_t = math.ceil(t * (2 / 3))
    k = math.ceil(t * (1 / 3))
    
    s = np.mean(vol[:k + 1])  
    res = np.zeros(t, dtype=np.float64)  
    res[0] = (y[0] - y_mean) / s  
    
    for i in range(1, t):
        res[i] = (y[i] - y_mean) / s  
        s = alpha * vol[i] + (1 - alpha) * s  
    
    return res


In [86]:

@cuda.jit
def dd_res_gpu(y, alpha, vol, s, s_0, res, y_mean, t):
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    
    if idx == 0:
        s[0] = s_0  
        res[0] = (y[0] - y_mean) / s[0] 

    cuda.syncthreads()

    if idx < t:
        for i in range(idx, t):
            if i > 0:
                res[i] = (y[i] - y_mean) / s[i - 1]  
                s[i] = alpha * vol[i] + (1 - alpha) * s[i - 1]              
            cuda.syncthreads()  


In [87]:
def gpu_dd_res(y):
    y = np.array(y, dtype=np.float64)
    t = len(y)

    y_mean = np.mean(y)
    rho = np.corrcoef(y - y_mean, np.sign(y - y_mean))[0, 1]
    vol = np.abs(y - y_mean) / rho
    alpha, _ = gpu_alpha(vol)  
    k = math.ceil(t * (1 / 3))
    s_0 = np.mean(vol[:k+1])

    y_device = cuda.to_device(y)
    vol_device = cuda.to_device(vol)
    res_device = cuda.device_array(t, dtype=np.float64)
    s_device = cuda.device_array(t, dtype=np.float64)
    alpha_device = cuda.to_device(np.array([alpha], dtype=np.float64))
    y_mean_device = cuda.to_device(np.array([y_mean], dtype=np.float64))

    device = cuda.get_current_device()
    max_threads_per_block = device.MAX_THREADS_PER_BLOCK
    threads_per_block = min(256, max_threads_per_block)
    blocks_per_grid = math.ceil(t / threads_per_block)

    start_event = cuda.event()
    end_event = cuda.event()

    start_event.record()

    dd_res_gpu[blocks_per_grid, threads_per_block](
        y_device, alpha_device[0], vol_device, s_device, s_0, res_device, y_mean_device[0], t
    )

    end_event.record()
    cuda.synchronize()

    elapsed_time_ms = cuda.event_elapsed_time(start_event, end_event)

    res = res_device.copy_to_host()

    return res, elapsed_time_ms


In [88]:
import time
start=time.time()
print(dd_res_cpu(cpu_results_df.AAPL))
print("Elapsed time on CPU: ",(time.time()-start), 'sec')

[-1.43807855e+00  3.21753949e-01 -1.50173452e+00  8.87305108e-01
 -2.44586734e-01 -8.41808037e-01 -9.90924690e-01  1.95791804e+00
  6.81773524e-01  6.28408910e-01  4.72305997e-01  1.13849344e+00
 -2.57603454e-01 -6.97550288e-01  9.94375772e-01 -2.44276312e-01
  2.93112834e-02 -1.24956117e+00  4.08953784e-01 -5.14439454e-01
 -1.86315315e-01  8.57655609e-01  8.48483738e-01  1.02832883e+00
  1.79935185e-01 -2.80528164e-01  1.98780533e-01 -6.94627249e-01
  1.23720045e+00  5.47307001e-01 -6.66781549e-01  1.78458020e-01
 -5.11352080e-01  4.85829929e-01  4.60250138e-01 -1.03697915e-01
 -7.37481279e-01  5.64689286e-01  3.00151968e-01  2.74657493e+00
 -1.08386946e+00  9.29655755e-01 -2.55351381e-01 -3.37075016e-01
 -6.62794181e-02  1.02384149e-01  6.12355750e-01 -6.09521880e-01
 -1.05221235e+00  3.96512445e-01  9.41817683e-01  6.01425023e-01
 -2.38623267e-01  1.95035831e+00  1.32455136e+00 -2.95018115e-01
 -2.75360791e-01  1.81139497e-01  6.10036301e-03 -3.80267521e-01
 -1.41635983e+00  1.60319

In [89]:
value,time=gpu_dd_res(gpu_results_df.AAPL)
print(value);
print(time/1000)

[-1.43807855e+00  3.21753949e-01 -1.50991487e+00  8.81693052e-01
 -2.42426929e-01 -8.40007608e-01 -9.86736319e-01  1.94171573e+00
  6.65119937e-01  6.13717939e-01  4.62067767e-01  1.11803633e+00
 -2.51658341e-01 -6.86071539e-01  9.78544933e-01 -2.39557457e-01
  2.89436730e-02 -1.24620189e+00  4.04718581e-01 -5.11343754e-01
 -1.85702145e-01  8.61289346e-01  8.49837766e-01  1.02753023e+00
  1.78935747e-01 -2.81110883e-01  2.00396277e-01 -7.05334092e-01
  1.25539056e+00  5.50510142e-01 -6.71949791e-01  1.79845667e-01
 -5.19220598e-01  4.94363534e-01  4.69497308e-01 -1.06079746e-01
 -7.60945990e-01  5.81458967e-01  3.09324574e-01  2.84530613e+00
 -1.08731917e+00  9.27210985e-01 -2.53838797e-01 -3.37272397e-01
 -6.66633235e-02  1.03940421e-01  6.27084787e-01 -6.24345215e-01
 -1.07812089e+00  4.03585104e-01  9.62356070e-01  6.11601216e-01
 -2.42827100e-01  1.99754063e+00  1.33006657e+00 -2.93521596e-01
 -2.75591986e-01  1.82404392e-01  6.18915240e-03 -3.89781867e-01
 -1.45760349e+00  1.62942

In [90]:
import time
start_time_cpu = time.time()
cpu_results = dd_res_cpu(cpu_results_df.AAPL)
cpu_time = time.time() - start_time_cpu

gpu_results, gpu_time = gpu_dd_res(gpu_results_df.AAPL)

is_valid = True
discrepancies = []

for i in range(len(cpu_results)):
    if not np.isclose(cpu_results[i], gpu_results[i], atol=1.0):
        is_valid = False
        discrepancies.append((i, cpu_results[i], gpu_results[i]))

print(is_valid)
if is_valid:
    print('GPU and CPU results match')
else:
    print('GPU and CPU results don\'t match')
    
print( {
    "cpu_results": cpu_results,
    "gpu_results": gpu_results,
    "cpu_time_seconds": cpu_time,
    "gpu_time_seconds": gpu_time / 1000,
    "comparison_valid": is_valid,  
})

[print(i) for i in discrepancies]


True
GPU and CPU results match
{'cpu_results': array([-1.43807855e+00,  3.21753949e-01, -1.50173452e+00,  8.87305108e-01,
       -2.44586734e-01, -8.41808037e-01, -9.90924690e-01,  1.95791804e+00,
        6.81773524e-01,  6.28408910e-01,  4.72305997e-01,  1.13849344e+00,
       -2.57603454e-01, -6.97550288e-01,  9.94375772e-01, -2.44276312e-01,
        2.93112834e-02, -1.24956117e+00,  4.08953784e-01, -5.14439454e-01,
       -1.86315315e-01,  8.57655609e-01,  8.48483738e-01,  1.02832883e+00,
        1.79935185e-01, -2.80528164e-01,  1.98780533e-01, -6.94627249e-01,
        1.23720045e+00,  5.47307001e-01, -6.66781549e-01,  1.78458020e-01,
       -5.11352080e-01,  4.85829929e-01,  4.60250138e-01, -1.03697915e-01,
       -7.37481279e-01,  5.64689286e-01,  3.00151968e-01,  2.74657493e+00,
       -1.08386946e+00,  9.29655755e-01, -2.55351381e-01, -3.37075016e-01,
       -6.62794181e-02,  1.02384149e-01,  6.12355750e-01, -6.09521880e-01,
       -1.05221235e+00,  3.96512445e-01,  9.41817683e

[]

# Residual matrix calculation

In [91]:
def construct_matrix_of_residuals(cpu_returns=cpu_results_df,gpu_returns=gpu_results_df,gpu_ret=calculate_returns_gpu,calculate_returns_cpu=calculate_returns_cpu,gpu_res=gpu_dd_res,cpu_res=dd_res_cpu,cpu_corr=cpu_correlation,gpu_corr=gpu_correlation):
    import time

   

    gpu_residuals=pd.DataFrame(columns=gpu_returns.columns)
    total_gpu_time=0
    for col in gpu_returns.columns:
        gpu_residuals[col],time=gpu_res(gpu_returns.loc[:,col])
        total_gpu_time+=(time/1000)
        
    cpu_residuals=pd.DataFrame(columns=cpu_returns.columns)
    total_cpu_time=0
    
    for col in cpu_returns.columns:
        import time
        start=time.time()
        cpu_residuals[col]=cpu_res(cpu_returns.loc[:,col])
        total_cpu_time+=(time.time()-start)

    print('GPU execution time for calculating residual matrix: ',total_gpu_time,' seconds')
    print('CPU execution time for calculating residual matrix: ',total_cpu_time,' seconds')

    
    col=[]
    for i in cpu_residuals.columns:
        if not np.allclose(cpu_residuals.loc[:,i].values, gpu_residuals.loc[:,i].values, atol=1.3):
            col.append(i)
    
    print(len(cpu_residuals.columns))
    
    returns_cpu=cpu_returns.drop(columns=col)
    returns_gpu=gpu_returns.drop(columns=col)
    
    cpu_residuals=cpu_residuals.drop(columns=col)
    gpu_residuals=gpu_residuals.drop(columns=col)

    print(returns_cpu.shape,returns_gpu.shape,cpu_residuals.shape,gpu_residuals.shape)
    cpu_corr_df=cpu_corr(returns_cpu)[0]
    cpu_corr_df=pd.DataFrame(cpu_corr_df,index=returns_cpu.columns,columns=returns_cpu.columns)
    gpu_corr_df=gpu_corr(returns_gpu)[0]
    gpu_corr_df=pd.DataFrame(gpu_corr_df,index=returns_gpu.columns,columns=returns_gpu.columns)

    if np.all(np.abs(cpu_corr_df-gpu_corr_df)>1e-4):
        print('GPU and CPU linear returns based correlations don\'t match')
        linear_correlations.pop()
        linear_correlations.append(gpu_corr_df)
    else:
        print('GPU and CPU linear returns based correlations match')
    
    
    col=[]
    for i in cpu_residuals.columns:
        if not np.allclose(cpu_residuals.loc[:,i].values, gpu_residuals.loc[:,i].values, atol=1.3):
            print(f"GPU and CPU residual matrix calculation results do not match!Difference is {np.max(np.abs(cpu_residuals.loc[:,i]-gpu_residuals.loc[:,i]))}");
            col.append(i)
    if len(col)==0:
        print(f"GPU and CPU residual matrix calculation results match! ");
    
    print(len(cpu_residuals.columns))
    
    #Calculating GPU Correlation
    gpu_corr_matrix,gpu_corr_time=gpu_corr(gpu_residuals);
    gpu_corr_matrix=pd.DataFrame(gpu_corr_matrix, columns=gpu_residuals.columns, index=gpu_residuals.columns)    
    print('Correlation matrix calculation time on GPU: ',gpu_corr_time/1000,'seconds')
    
    #Calculating CPU Correlation
    cpu_corr_matrix,cpu_corr_time=cpu_corr(cpu_residuals);
    cpu_corr_matrix=pd.DataFrame(cpu_corr_matrix,columns=cpu_residuals.columns, index=cpu_residuals.columns)
    print('Correlation matrix calculation time on CPUL ',cpu_corr_time,' seconds')

    
    #Matching the results of the correlation calculations
    diff = np.abs(np.array(cpu_corr_matrix) - np.array(gpu_corr_matrix))
    
    if np.all(diff < 1e-1):
        print(f"The correlation results from CPU and GPU are consistent within the threshold. The max difference is: {np.max(diff)}")
    else:
        print(f"The correlation results differ beyond the threshold. Max difference: {np.max(diff)}")
    
    #print(cpu_corr_matrix.shape,gpu_corr_matrix.shape)
    return returns_cpu,returns_gpu,cpu_corr_df,gpu_corr_df, cpu_corr_matrix, gpu_corr_matrix

In [92]:
returns_cpu,returns_gpu,cpu_corr_df, gpu_corr_df,cpu_res_correlation,gpu_res_correlation=construct_matrix_of_residuals(cpu_results_df,gpu_results_df)
residual_linear_correlations.append(gpu_res_correlation)

GPU execution time for calculating residual matrix:  0.6249540180265982  seconds
CPU execution time for calculating residual matrix:  23.652461290359497  seconds
1654
(251, 1648) (251, 1648) (251, 1648) (251, 1648)
GPU and CPU linear returns based correlations match
GPU and CPU residual matrix calculation results match! 
1648
Correlation matrix calculation time on GPU:  0.03320729446411133 seconds
Correlation matrix calculation time on CPUL  18.87254548072815  seconds
The correlation results from CPU and GPU are consistent within the threshold. The max difference is: 0.05537992715835571


In [93]:
returns_cpu

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
0,-0.076389,0.004633,0.027027,0.041667,-0.084310,-0.004201,-0.011628,-0.071429,0.000000,0.003077,...,-0.021053,0.000000,-0.046512,0.000000,-0.006885,-0.020100,-0.014428,-0.057692,0.000000,-0.048423
1,-0.062030,0.057648,0.026316,0.008889,0.014633,0.004219,0.000000,0.000000,0.000000,-0.012269,...,0.000000,0.017287,0.051491,-0.007353,0.005200,0.005127,0.019144,-0.040816,0.000000,-0.001183
2,-0.038076,-0.013081,-0.051282,0.004405,-0.086538,0.016807,0.000000,0.153846,0.000000,0.003106,...,0.010753,0.001307,-0.020618,0.000000,-0.006897,-0.008503,-0.054144,0.053191,-0.025974,0.014218
3,0.083333,-0.002946,0.027027,-0.035088,0.047369,0.033058,0.011765,0.000000,0.000000,0.003095,...,-0.042553,-0.002611,0.023684,0.000000,0.010417,-0.022298,-0.011098,-0.010101,0.066666,0.002337
4,0.060577,-0.002954,0.000000,0.000000,-0.017588,-0.018000,-0.023256,0.600000,0.000000,0.000000,...,0.044444,0.023560,-0.023136,0.000000,-0.006873,0.040351,0.033668,0.010204,0.025000,-0.018066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.064476,0.096456,0.066667,-0.006897,0.066667,0.002670,0.036232,0.051282,0.000000,0.044716,...,0.090226,0.039049,0.051948,-0.007195,-0.004238,-0.007767,0.025807,-0.142858,0.000000,0.060085
247,-0.021714,-0.007181,-0.093750,-0.020833,-0.020832,0.000000,0.006993,-0.048780,0.000000,-0.035019,...,0.048276,0.000170,-0.037037,-0.043478,-0.006383,-0.007828,-0.026730,-0.250001,0.014706,-0.004048
248,0.045561,-0.010850,0.000000,0.014185,0.008510,0.038615,0.041666,0.012821,0.074626,-0.002017,...,0.003289,0.060557,0.025641,0.046153,-0.008565,0.019724,0.033926,0.222223,0.000000,0.002032
249,-0.011174,0.012797,0.000000,-0.013986,0.000000,0.021795,-0.006756,-0.012658,0.000000,0.002021,...,-0.022951,0.052469,0.000000,-0.029412,0.084233,0.034816,0.098437,-0.181819,-0.043479,0.007100


In [94]:
returns_gpu

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
0,-0.076389,0.004633,0.027027,0.041667,-0.084310,-0.004201,-0.011628,-0.071429,0.000000,0.003077,...,-0.021053,0.000000,-0.046512,0.000000,-0.006885,-0.020100,-0.014428,-0.057692,0.000000,-0.048423
1,-0.062030,0.057648,0.026316,0.008889,0.014633,0.004219,0.000000,0.000000,0.000000,-0.012269,...,0.000000,0.017287,0.051491,-0.007353,0.005200,0.005127,0.019144,-0.040816,0.000000,-0.001183
2,-0.038076,-0.013081,-0.051282,0.004405,-0.086538,0.016807,0.000000,0.153846,0.000000,0.003106,...,0.010753,0.001307,-0.020618,0.000000,-0.006897,-0.008503,-0.054144,0.053191,-0.025974,0.014218
3,0.083333,-0.002946,0.027027,-0.035088,0.047369,0.033058,0.011765,0.000000,0.000000,0.003095,...,-0.042553,-0.002611,0.023684,0.000000,0.010417,-0.022298,-0.011098,-0.010101,0.066666,0.002337
4,0.060577,-0.002954,0.000000,0.000000,-0.017588,-0.018000,-0.023256,0.600000,0.000000,0.000000,...,0.044444,0.023560,-0.023136,0.000000,-0.006873,0.040351,0.033668,0.010204,0.025000,-0.018066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.064476,0.096456,0.066667,-0.006897,0.066667,0.002670,0.036232,0.051282,0.000000,0.044716,...,0.090226,0.039049,0.051948,-0.007195,-0.004238,-0.007767,0.025807,-0.142858,0.000000,0.060085
247,-0.021714,-0.007181,-0.093750,-0.020833,-0.020832,0.000000,0.006993,-0.048780,0.000000,-0.035019,...,0.048276,0.000170,-0.037037,-0.043478,-0.006383,-0.007828,-0.026730,-0.250001,0.014706,-0.004048
248,0.045561,-0.010850,0.000000,0.014185,0.008510,0.038615,0.041666,0.012821,0.074626,-0.002017,...,0.003289,0.060557,0.025641,0.046153,-0.008565,0.019724,0.033926,0.222223,0.000000,0.002032
249,-0.011174,0.012797,0.000000,-0.013986,0.000000,0.021795,-0.006756,-0.012658,0.000000,0.002021,...,-0.022951,0.052469,0.000000,-0.029412,0.084233,0.034816,0.098437,-0.181819,-0.043479,0.007100


In [95]:
cpu_returns[-1]=returns_cpu
gpu_returns[-1]=returns_gpu
cpu_linear_correlations[-1]=cpu_corr_df

In [96]:
#linear_correlations.pop()
linear_correlations.append(gpu_corr_df)
print(cpu_corr_df.shape)
print(gpu_corr_df.shape)
print(cpu_res_correlation.shape)
print(gpu_res_correlation.shape)

(1648, 1648)
(1648, 1648)
(1648, 1648)
(1648, 1648)


In [97]:
cpu_res_correlation

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
A,1.000000,0.066347,0.094327,0.078391,0.176225,0.156762,0.109512,0.131874,-0.020365,0.182531,...,0.233659,0.020782,0.136605,-0.020880,0.056275,0.164255,0.014367,0.103502,0.154302,0.128199
AA,0.066347,1.000000,0.036923,-0.051139,-0.075501,0.165870,0.185552,-0.053621,0.036349,0.042068,...,0.153735,0.148244,0.084193,-0.016036,0.155964,0.176955,-0.043821,0.017328,-0.062412,0.188717
AAME,0.094327,0.036923,1.000000,-0.031525,0.002350,0.058176,0.016582,-0.022911,0.079924,0.152849,...,-0.008297,-0.076368,0.006464,0.095190,0.054913,-0.008410,-0.021817,0.023650,0.011470,0.007662
AAON,0.078391,-0.051139,-0.031525,1.000000,0.025097,0.046178,-0.002818,0.060750,0.003455,0.015929,...,0.042471,0.171254,-0.022072,-0.001603,-0.042076,-0.085604,0.111332,-0.008575,-0.063355,0.002345
AAPL,0.176225,-0.075501,0.002350,0.025097,1.000000,0.100317,-0.070849,0.045370,-0.030591,0.091119,...,0.174528,0.021091,0.172913,0.084198,-0.053482,0.024807,0.208229,-0.001044,0.029829,0.111241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.164255,0.176955,-0.008410,-0.085604,0.024807,0.249229,0.047294,0.092424,0.043823,0.003965,...,-0.000101,0.086324,0.175677,0.049951,0.085770,1.000000,0.058616,-0.027919,0.142728,0.165330
ZBRA,0.014367,-0.043821,-0.021817,0.111332,0.208229,0.155694,0.072557,0.116183,0.032051,0.200500,...,0.050730,0.049298,0.099617,0.051629,0.014106,0.058616,1.000000,-0.059476,0.006786,0.125324
ZD,0.103502,0.017328,0.023650,-0.008575,-0.001044,0.009075,0.047240,0.082585,0.117025,0.109884,...,0.073342,0.059568,-0.007168,0.074053,0.061419,-0.027919,-0.059476,1.000000,0.047868,-0.056933
ZEUS,0.154302,-0.062412,0.011470,-0.063355,0.029829,0.005367,0.012654,-0.070827,0.099143,0.049107,...,-0.070489,0.069548,0.133373,-0.049016,0.024198,0.142728,0.006786,0.047868,1.000000,0.064631


In [98]:
gpu_res_correlation

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
A,1.000000,0.066347,0.094327,0.078391,0.178311,0.156762,0.112979,0.131440,-0.020365,0.182531,...,0.224442,0.020782,0.136605,-0.020880,0.056275,0.162292,0.014367,0.103502,0.154302,0.128199
AA,0.066347,1.000000,0.036923,-0.051139,-0.073258,0.165870,0.179104,-0.051631,0.036349,0.042068,...,0.157273,0.148244,0.084193,-0.016036,0.155964,0.174304,-0.043821,0.017328,-0.062412,0.188717
AAME,0.094327,0.036923,1.000000,-0.031525,0.002739,0.058176,0.020799,-0.022160,0.079924,0.152849,...,-0.010618,-0.076368,0.006464,0.095190,0.054913,-0.008427,-0.021817,0.023650,0.011470,0.007662
AAON,0.078391,-0.051139,-0.031525,1.000000,0.023038,0.046178,-0.001665,0.059203,0.003455,0.015929,...,0.039745,0.171254,-0.022072,-0.001603,-0.042076,-0.088448,0.111332,-0.008575,-0.063355,0.002345
AAPL,0.178311,-0.073258,0.002739,0.023038,1.000000,0.099495,-0.062512,0.049117,-0.030630,0.093068,...,0.174004,0.021786,0.172884,0.082878,-0.053752,0.026332,0.208445,-0.001155,0.029132,0.111660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.162292,0.174304,-0.008427,-0.088448,0.026332,0.246532,0.046837,0.093609,0.043427,0.003925,...,-0.000166,0.084842,0.177214,0.050180,0.084705,1.000000,0.059546,-0.028253,0.143292,0.163973
ZBRA,0.014367,-0.043821,-0.021817,0.111332,0.208445,0.155694,0.075714,0.115612,0.032051,0.200500,...,0.050531,0.049298,0.099617,0.051629,0.014106,0.059546,1.000000,-0.059476,0.006786,0.125324
ZD,0.103502,0.017328,0.023650,-0.008575,-0.001155,0.009075,0.050008,0.084359,0.117025,0.109884,...,0.067757,0.059568,-0.007168,0.074053,0.061419,-0.028253,-0.059476,1.000000,0.047868,-0.056933
ZEUS,0.154302,-0.062412,0.011470,-0.063355,0.029132,0.005367,0.011850,-0.071324,0.099143,0.049107,...,-0.070630,0.069548,0.133373,-0.049016,0.024198,0.143292,0.006786,0.047868,1.000000,0.064631


In [99]:
cpu_corr_df

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
A,1.000000,0.060055,0.105745,0.068161,0.187420,0.163134,0.112485,0.149201,-0.020993,0.178850,...,0.265733,0.024445,0.137632,-0.016805,0.063693,0.161885,0.039665,0.078390,0.151110,0.137440
AA,0.060055,1.000000,0.040147,-0.052112,-0.073040,0.178934,0.162735,-0.059906,0.033063,0.038447,...,0.132752,0.156911,0.092623,-0.032667,0.167220,0.197395,-0.023940,0.019908,-0.065141,0.189320
AAME,0.105745,0.040147,1.000000,-0.027005,-0.003294,0.063666,0.025925,-0.023721,0.085917,0.147374,...,0.007508,-0.081942,0.000194,0.086593,0.075938,-0.006023,-0.009842,0.071114,-0.051990,-0.053066
AAON,0.068161,-0.052112,-0.027005,1.000000,0.019104,0.036082,-0.003111,0.046418,-0.006045,0.011403,...,0.024203,0.162109,-0.024780,0.004654,-0.045923,-0.080574,0.106002,0.035837,-0.066699,-0.033045
AAPL,0.187420,-0.073040,-0.003294,0.019104,1.000000,0.121327,-0.046102,0.011134,-0.026474,0.091759,...,0.185977,0.015197,0.172271,0.089980,-0.056203,0.031793,0.239782,0.010591,0.030175,0.122204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.161885,0.197395,-0.006023,-0.080574,0.031793,0.260947,0.053375,0.080772,0.044335,-0.004758,...,-0.009276,0.096250,0.158490,0.038880,0.082647,1.000000,0.075392,-0.041410,0.138217,0.225962
ZBRA,0.039665,-0.023940,-0.009842,0.106002,0.239782,0.159299,0.086417,0.091207,0.033217,0.192230,...,0.038478,0.068191,0.113366,0.038284,0.006531,0.075392,1.000000,-0.080805,0.037998,0.100075
ZD,0.078390,0.019908,0.071114,0.035837,0.010591,0.012549,0.102757,0.069893,0.188665,0.081197,...,0.076257,0.045094,-0.016395,0.115815,0.023252,-0.041410,-0.080805,1.000000,0.039170,-0.039563
ZEUS,0.151110,-0.065141,-0.051990,-0.066699,0.030175,0.024711,-0.009316,-0.018788,0.099016,0.037436,...,-0.037441,0.054530,0.124562,-0.052521,0.003075,0.138217,0.037998,0.039170,1.000000,0.086746


In [100]:
gpu_corr_df

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
A,1.000000,0.060055,0.105745,0.068161,0.187420,0.163134,0.112485,0.149201,-0.020993,0.178850,...,0.265733,0.024445,0.137632,-0.016805,0.063693,0.161885,0.039665,0.078390,0.151110,0.137440
AA,0.060055,1.000000,0.040147,-0.052112,-0.073040,0.178934,0.162735,-0.059906,0.033063,0.038447,...,0.132752,0.156911,0.092623,-0.032667,0.167220,0.197395,-0.023940,0.019908,-0.065141,0.189320
AAME,0.105745,0.040147,1.000000,-0.027005,-0.003294,0.063666,0.025925,-0.023721,0.085917,0.147374,...,0.007508,-0.081942,0.000194,0.086593,0.075938,-0.006023,-0.009842,0.071114,-0.051990,-0.053066
AAON,0.068161,-0.052112,-0.027005,1.000000,0.019104,0.036082,-0.003111,0.046418,-0.006045,0.011403,...,0.024203,0.162109,-0.024780,0.004654,-0.045923,-0.080574,0.106002,0.035837,-0.066699,-0.033045
AAPL,0.187420,-0.073040,-0.003294,0.019104,1.000000,0.121327,-0.046102,0.011134,-0.026474,0.091759,...,0.185977,0.015197,0.172271,0.089980,-0.056203,0.031793,0.239782,0.010591,0.030175,0.122205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.161885,0.197395,-0.006023,-0.080574,0.031793,0.260947,0.053375,0.080772,0.044335,-0.004757,...,-0.009276,0.096250,0.158490,0.038880,0.082647,1.000000,0.075392,-0.041410,0.138217,0.225962
ZBRA,0.039665,-0.023940,-0.009842,0.106002,0.239782,0.159299,0.086417,0.091207,0.033217,0.192230,...,0.038478,0.068191,0.113366,0.038284,0.006531,0.075392,1.000000,-0.080805,0.037998,0.100075
ZD,0.078390,0.019908,0.071114,0.035837,0.010591,0.012549,0.102757,0.069893,0.188665,0.081197,...,0.076257,0.045094,-0.016395,0.115815,0.023252,-0.041410,-0.080805,1.000000,0.039170,-0.039563
ZEUS,0.151110,-0.065141,-0.051990,-0.066699,0.030175,0.024711,-0.009316,-0.018788,0.099016,0.037436,...,-0.037441,0.054530,0.124562,-0.052521,0.003075,0.138217,0.037998,0.039170,1.000000,0.086746


In [101]:
print(linear_correlations[-1].shape)

(1648, 1648)


In [102]:
print(linear_correlations[-1].shape)
print(cpu_linear_correlations[-1].shape)
print(cpu_returns[-1].shape)
print(gpu_returns[-1].shape)

(1648, 1648)
(1648, 1648)
(251, 1648)
(251, 1648)


In [103]:
len(cpu_linear_correlations)

1

In [104]:
cpu_linear_correlations.append(cpu_corr_df)

In [105]:
cpu_corr_df.shape

(1648, 1648)

In [106]:
gpu_corr_df.shape

(1648, 1648)

In [107]:
import time

st=time.time()
vol={}
c=1
for i in returns_cpu.columns:
    print(c,i)
    vol[i]=dd_vol_cpu(returns_cpu.loc[:,i])
    c+=1
tt=time.time()-st

1 A
2 AA
3 AAME
4 AAON
5 AAPL
6 AB
7 ABCB
8 ABEO
9 ABEV
10 ABM
11 ABT
12 ACGL
13 ACHC
14 ACHV
15 ACIW
16 ACNB
17 ACNT
18 ACU
19 ADBE
20 ADC
21 ADI
22 ADM
23 ADP
24 ADSK
25 ADTN
26 AE
27 AEE
28 AEG
29 AEHR
30 AEIS
31 AEM
32 AEMD
33 AEO
34 AEP
35 AES
36 AFG
37 AFL
38 AGCO
39 AGEN
40 AGM
41 AGX
42 AGYS
43 AIFF
44 AIG
45 AIM
46 AIN
47 AIOT
48 AIR
49 AIRT
50 AIT
51 AJG
52 AKAM
53 AKR
54 ALB
55 ALCO
56 ALE
57 ALG
58 ALK
59 ALKS
60 ALL
61 ALNT
62 ALOT
63 ALTS
64 ALV
65 ALX
66 AMAT
67 AMD
68 AME
69 AMED
70 AMG
71 AMGN
72 AMKR
73 AMRN
74 AMS
75 AMSC
76 AMT
77 AMWD
78 AMZN
79 AN
80 ANDE
81 ANF
82 ANIK
83 ANIP
84 ANIX
85 ANSS
86 AON
87 AORT
88 AOS
89 AP
90 APA
91 APD
92 APH
93 APOG
94 APT
95 APTO
96 ARCB
97 ARE
98 ARKR
99 ARL
100 ARLP
101 ARMP
102 AROW
103 ARTNA
104 ARTW
105 ARW
106 ASA
107 ASB
108 ASGN
109 ASH
110 ASML
111 ASRT
112 ASRV
113 ASTC
114 ASTE
115 ASUR
116 ASYS
117 ATGE
118 ATI
119 ATLC
120 ATLO
121 ATNI
122 ATO
123 ATR
124 ATRO
125 AU
126 AUB
127 AUBN
128 AUDC
129 AVA
130 AVB
131 AVD

In [108]:
#vol['NEN']

In [109]:
returns_cpu.iloc[:,[0]].columns[0]

'A'

# Dynamic Correlation

In [110]:
def dynamic_cov_cpu(residuals_corr,returns,dd_vol_cpu=dd_vol_cpu,tt=tt):
    import time
    
    dataframe=np.zeros((returns.shape[1],returns.shape[1]))
    residuals=np.array(residuals_corr)

    start=time.time()
    
    for i in range(returns.shape[1]):
        print(i)
        vol_i=vol[returns.iloc[:,[i]].columns[0]]
        for j in range(returns.shape[1]):
            vol_j=vol[returns.iloc[:,[j]].columns[0]]
            res=residuals[i,j]
            dataframe[i,j]=vol_i*vol_j*res
    total_time=time.time()-start
    dataframe=pd.DataFrame(dataframe)
    dataframe.columns = returns.columns
    dataframe.index = returns.columns

    return dataframe, total_time+tt

In [111]:
@cuda.jit
def dynamic_cov_gpu(residuals, vol_device, result_device, n):
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    if idx < n:
        for j in range(n):
            result_device[idx, j] = residuals[idx, j] * vol_device[idx] * vol_device[j]

In [112]:
def gpu_dynamic_cov(residuals_corr, returns, gpu_dd_vol):
    residuals_corr = np.array(residuals_corr)
    cols=returns.columns
    returns = np.array(returns)

    t, n = returns.shape  

    # Ensure vol is created as a numpy array of dtype float64
    vol = np.array([gpu_dd_vol(returns[:, j])[0] for j in range(returns.shape[1])], dtype=np.float64)

    # Create device arrays with dtype=float64 to avoid type mismatch
    vol_device = cuda.to_device(vol)  
    result_device = cuda.device_array((n, n), dtype=np.float64)  
    residuals_device = cuda.to_device(residuals_corr)
    returns_device = cuda.to_device(returns)

    device = cuda.get_current_device()
    max_threads_per_block = device.MAX_THREADS_PER_BLOCK
    threads_per_block = min(256, max_threads_per_block)
    blocks_per_grid = math.ceil(n / threads_per_block)+1

    start_event = cuda.event()
    end_event = cuda.event()
    
    start_event.record()

    # Launch the kernel
    dynamic_cov_gpu[blocks_per_grid, threads_per_block](
        residuals_device, vol_device, result_device, n
    )

    end_event.record()
    cuda.synchronize()

    elapsed_time_ms_cov = cuda.event_elapsed_time(start_event, end_event)

    result = result_device.copy_to_host()

    result_df = pd.DataFrame(result)
    result_df.columns = cols
    result_df.index = cols

    return result_df, elapsed_time_ms_cov

In [113]:
dynamic_cov_cor_cpu,total_time=dynamic_cov_cpu(cpu_res_correlation,returns_cpu)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [114]:
print('CPU execution time of the dynamic covariance calculation is: ', total_time)

CPU execution time of the dynamic covariance calculation is:  282.25103402137756


In [115]:
dynamic_cov_cor_gpu,elapsed_time=gpu_dynamic_cov(gpu_res_correlation,returns_gpu,gpu_dd_vol)

/home/cs/grad/dasj/anaconda3/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/cs/grad/dasj/anaconda3/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 8 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [116]:
print('GPU execution time of the dynamic covariance calculation is: ', elapsed_time/1000)

GPU execution time of the dynamic covariance calculation is:  0.04591481781005859


In [117]:
diff = np.abs(dynamic_cov_cor_cpu - dynamic_cov_cor_gpu)

if np.all(diff < 1e-2):
    print(f"The results from CPU and GPU are consistent within the threshold. Max difference: {np.max(diff)}")
    dynamic_correlations.append(dynamic_cov_cor_gpu)
else:
    print(f"The results differ beyond the threshold. Max difference: {np.max(diff)}")

The results from CPU and GPU are consistent within the threshold. Max difference: 0.008508196363951809


In [118]:
def match_and_filter_correlation_matrices(cpu_corr_matrix, gpu_corr_matrix, threshold=0.05):
    if not cpu_corr_matrix.index.equals(cpu_corr_matrix.columns) or not gpu_corr_matrix.index.equals(gpu_corr_matrix.columns):
        raise ValueError("Correlation matrices must be square and have the same index and column names.")
    
    if not cpu_corr_matrix.shape == gpu_corr_matrix.shape:
        raise ValueError("CPU and GPU correlation matrices must have the same dimensions.")

    diff = np.abs(cpu_corr_matrix.values - gpu_corr_matrix.values)
    mismatched_indices = np.argwhere(diff > threshold)

    discarded_stocks = set()
    for i, j in mismatched_indices:
        stock_row = cpu_corr_matrix.index[i]
        stock_col = cpu_corr_matrix.columns[j]
        discarded_stocks.add(stock_row)
        discarded_stocks.add(stock_col)
    
    discarded_stocks = sorted(discarded_stocks)
    filtered_cpu_corr = cpu_corr_matrix.drop(index=discarded_stocks, columns=discarded_stocks)
    filtered_gpu_corr = gpu_corr_matrix.drop(index=discarded_stocks, columns=discarded_stocks)
    
    return filtered_cpu_corr, filtered_gpu_corr, discarded_stocks
    
filtered_dyn_cov_cpu,filtered_dyn_cov_gpu,dis_stocks=match_and_filter_correlation_matrices(dynamic_cov_cor_cpu,dynamic_cov_cor_gpu)



if all(stock in cpu_returns[-1].columns for stock in dis_stocks):
    gpu_returns[-1] = gpu_returns[-1].drop( columns=dis_stocks)
    cpu_returns[-1] = cpu_returns[-1].drop( columns=dis_stocks)

if all(stock in linear_correlations[-1].columns for stock in dis_stocks):
    linear_correlations[-1]=linear_correlations[-1].drop(index=dis_stocks, columns=dis_stocks)
    cpu_linear_correlations[-1] = cpu_linear_correlations[-1].drop(index=dis_stocks, columns=dis_stocks)




diff = np.abs(filtered_dyn_cov_cpu - filtered_dyn_cov_gpu)

if np.all(diff < 1e-1):
    print(f"The results from CPU and GPU are consistent within the threshold. Max difference: {np.max(diff)}")
    dynamic_correlations.clear()
    cpu_dynamic_correlations.clear()
    dynamic_correlations.append(filtered_dyn_cov_gpu)
    cpu_dynamic_correlations.append(filtered_dyn_cov_cpu)
else:
    print(f"The results differ beyond the threshold. Max difference: {np.max(diff)}")

The results from CPU and GPU are consistent within the threshold. Max difference: 0.008508196363951809


In [119]:
print(len(dis_stocks))
print(dis_stocks)

0
[]


In [120]:
filtered_dyn_cov_gpu.shape

(1648, 1648)

In [121]:
filtered_dyn_cov_cpu.shape

(1648, 1648)

In [122]:
#cpu_dynamic_correlations.append(dynamic_cov_cor_cpu)

In [123]:
print(len(linear_correlations))
print(len(cpu_linear_correlations))
print(len(dynamic_correlations))
print(len(cpu_dynamic_correlations))

3
2
1
1


In [124]:
print(linear_correlations[-1].shape)
print(cpu_linear_correlations[-1].shape)
print(cpu_returns[-1].shape)
print(gpu_returns[-1].shape)
print(dynamic_correlations[-1].shape)
print(cpu_dynamic_correlations[-1].shape)

(1648, 1648)
(1648, 1648)
(251, 1648)
(251, 1648)
(1648, 1648)
(1648, 1648)


In [125]:
all(stock in linear_correlations[-1].columns for stock in dis_stocks)

True

In [126]:
cpu_linear_correlations[-1].index.name='ticker'
cpu_linear_correlations[-1].to_csv(os.path.join(os.getcwd(),'cpu_linear','linear_data_corr_cpu_'+str(year)+'.csv'))

In [127]:
linear_correlations[-1].index.name='ticker'
linear_correlations[-1].to_csv(os.path.join(os.getcwd(),'gpu_linear','linear_data_corr_gpu_'+str(year)+'.csv'))

In [128]:
pd.read_csv(os.path.join(os.getcwd(),'cpu_linear',f'linear_data_corr_cpu_{year}.csv'),index_col='ticker')

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
ticker,,,,,,,,,,,,,,,,,,,,,
A,1.000000,0.060055,0.105745,0.068161,0.187420,0.163134,0.112485,0.149201,-0.020993,0.178850,...,0.265733,0.024445,0.137632,-0.016805,0.063693,0.161885,0.039665,0.078390,0.151110,0.137440
AA,0.060055,1.000000,0.040147,-0.052112,-0.073040,0.178934,0.162735,-0.059906,0.033063,0.038447,...,0.132752,0.156911,0.092623,-0.032667,0.167220,0.197395,-0.023940,0.019908,-0.065141,0.189320
AAME,0.105745,0.040147,1.000000,-0.027005,-0.003294,0.063666,0.025925,-0.023721,0.085917,0.147374,...,0.007508,-0.081942,0.000194,0.086593,0.075938,-0.006023,-0.009842,0.071114,-0.051990,-0.053066
AAON,0.068161,-0.052112,-0.027005,1.000000,0.019104,0.036082,-0.003111,0.046418,-0.006045,0.011403,...,0.024203,0.162109,-0.024780,0.004654,-0.045923,-0.080574,0.106002,0.035837,-0.066699,-0.033045
AAPL,0.187420,-0.073040,-0.003294,0.019104,1.000000,0.121327,-0.046102,0.011134,-0.026474,0.091759,...,0.185977,0.015197,0.172271,0.089980,-0.056203,0.031793,0.239782,0.010591,0.030175,0.122204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.161885,0.197395,-0.006023,-0.080574,0.031793,0.260947,0.053375,0.080772,0.044335,-0.004758,...,-0.009276,0.096250,0.158490,0.038880,0.082647,1.000000,0.075392,-0.041410,0.138217,0.225962
ZBRA,0.039665,-0.023940,-0.009842,0.106002,0.239782,0.159299,0.086417,0.091207,0.033217,0.192230,...,0.038478,0.068191,0.113366,0.038284,0.006531,0.075392,1.000000,-0.080805,0.037998,0.100075
ZD,0.078390,0.019908,0.071114,0.035837,0.010591,0.012549,0.102757,0.069893,0.188665,0.081197,...,0.076257,0.045094,-0.016395,0.115815,0.023252,-0.041410,-0.080805,1.000000,0.039170,-0.039563


In [129]:
pd.read_csv(os.path.join(os.getcwd(),'gpu_linear',f'linear_data_corr_gpu_{year}.csv'),index_col='ticker')

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
ticker,,,,,,,,,,,,,,,,,,,,,
A,1.000000,0.060055,0.105745,0.068161,0.187420,0.163134,0.112485,0.149201,-0.020993,0.178850,...,0.265733,0.024445,0.137632,-0.016805,0.063693,0.161885,0.039665,0.078390,0.151110,0.137440
AA,0.060055,1.000000,0.040147,-0.052112,-0.073040,0.178934,0.162735,-0.059906,0.033063,0.038447,...,0.132752,0.156911,0.092623,-0.032667,0.167220,0.197395,-0.023940,0.019908,-0.065141,0.189320
AAME,0.105745,0.040147,1.000000,-0.027005,-0.003294,0.063666,0.025925,-0.023721,0.085917,0.147374,...,0.007508,-0.081942,0.000194,0.086593,0.075938,-0.006023,-0.009842,0.071114,-0.051990,-0.053066
AAON,0.068161,-0.052112,-0.027005,1.000000,0.019104,0.036082,-0.003111,0.046418,-0.006045,0.011403,...,0.024203,0.162109,-0.024780,0.004654,-0.045923,-0.080574,0.106002,0.035837,-0.066699,-0.033045
AAPL,0.187420,-0.073040,-0.003294,0.019104,1.000000,0.121327,-0.046102,0.011134,-0.026474,0.091759,...,0.185977,0.015197,0.172271,0.089980,-0.056203,0.031793,0.239782,0.010591,0.030175,0.122205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.161885,0.197395,-0.006023,-0.080574,0.031793,0.260947,0.053375,0.080772,0.044335,-0.004757,...,-0.009276,0.096250,0.158490,0.038880,0.082647,1.000000,0.075392,-0.041410,0.138217,0.225962
ZBRA,0.039665,-0.023940,-0.009842,0.106002,0.239782,0.159299,0.086417,0.091207,0.033217,0.192230,...,0.038478,0.068191,0.113366,0.038284,0.006531,0.075392,1.000000,-0.080805,0.037998,0.100075
ZD,0.078390,0.019908,0.071114,0.035837,0.010591,0.012549,0.102757,0.069893,0.188665,0.081197,...,0.076257,0.045094,-0.016395,0.115815,0.023252,-0.041410,-0.080805,1.000000,0.039170,-0.039563


In [130]:
dynamic_cov_cor_cpu

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
A,0.003817,0.000140,0.000252,0.000154,0.000529,0.000242,0.000194,0.000526,-0.000017,0.000215,...,1.152240e-03,0.000034,0.000529,-0.000033,5.717272e-05,0.000283,0.000041,0.001105,0.000429,0.000189
AA,0.000140,0.001166,0.000055,-0.000056,-0.000125,0.000142,0.000182,-0.000118,0.000016,0.000027,...,4.190781e-04,0.000134,0.000180,-0.000014,8.759106e-05,0.000168,-0.000070,0.000102,-0.000096,0.000153
AAME,0.000252,0.000055,0.001877,-0.000044,0.000005,0.000063,0.000021,-0.000064,0.000046,0.000126,...,-2.868960e-05,-0.000088,0.000018,0.000104,3.912068e-05,-0.000010,-0.000044,0.000177,0.000022,0.000008
AAON,0.000154,-0.000056,-0.000044,0.001017,0.000039,0.000037,-0.000003,0.000125,0.000001,0.000010,...,1.080929e-04,0.000145,-0.000044,-0.000001,-2.206220e-05,-0.000076,0.000166,-0.000047,-0.000091,0.000002
AAPL,0.000529,-0.000125,0.000005,0.000039,0.002360,0.000122,-0.000099,0.000142,-0.000020,0.000084,...,6.767085e-04,0.000027,0.000527,0.000104,-4.272223e-05,0.000034,0.000473,-0.000009,0.000065,0.000129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.000283,0.000168,-0.000010,-0.000076,0.000034,0.000174,0.000038,0.000166,0.000016,0.000002,...,-2.254811e-07,0.000064,0.000307,0.000035,3.931895e-05,0.000777,0.000076,-0.000134,0.000179,0.000110
ZBRA,0.000041,-0.000070,-0.000044,0.000166,0.000473,0.000182,0.000097,0.000351,0.000020,0.000179,...,1.892892e-04,0.000061,0.000292,0.000061,1.084336e-05,0.000076,0.002185,-0.000480,0.000014,0.000139
ZD,0.001105,0.000102,0.000177,-0.000047,-0.000009,0.000039,0.000235,0.000922,0.000268,0.000362,...,1.011558e-03,0.000273,-0.000078,0.000324,1.745225e-04,-0.000134,-0.000480,0.029861,0.000372,-0.000234
ZEUS,0.000429,-0.000096,0.000022,-0.000091,0.000065,0.000006,0.000016,-0.000206,0.000059,0.000042,...,-2.532287e-04,0.000083,0.000376,-0.000056,1.790968e-05,0.000179,0.000014,0.000372,0.002026,0.000069


In [131]:
dynamic_cov_cor_gpu

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
A,0.003817,0.000140,0.000253,0.000154,0.000505,0.000242,0.000172,0.000505,-0.000017,0.000215,...,1.073485e-03,0.000034,0.000529,-0.000033,5.721862e-05,0.000281,0.000041,0.001105,0.000429,0.000189
AA,0.000140,0.001165,0.000055,-0.000056,-0.000115,0.000142,0.000150,-0.000110,0.000016,0.000027,...,4.155465e-04,0.000134,0.000180,-0.000014,8.760381e-05,0.000167,-0.000070,0.000102,-0.000096,0.000153
AAME,0.000253,0.000055,0.001877,-0.000044,0.000005,0.000063,0.000022,-0.000060,0.000046,0.000126,...,-3.561604e-05,-0.000088,0.000018,0.000104,3.915804e-05,-0.000010,-0.000044,0.000177,0.000022,0.000008
AAON,0.000154,-0.000056,-0.000044,0.001017,0.000034,0.000037,-0.000001,0.000117,0.000001,0.000010,...,9.811480e-05,0.000145,-0.000044,-0.000001,-2.208081e-05,-0.000079,0.000166,-0.000047,-0.000091,0.000002
AAPL,0.000505,-0.000115,0.000005,0.000034,0.002100,0.000114,-0.000071,0.000140,-0.000019,0.000081,...,6.173111e-04,0.000026,0.000496,0.000096,-4.053883e-05,0.000034,0.000447,-0.000009,0.000060,0.000122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.000281,0.000167,-0.000010,-0.000079,0.000034,0.000173,0.000032,0.000163,0.000016,0.000002,...,-3.604159e-07,0.000063,0.000312,0.000036,3.910462e-05,0.000787,0.000078,-0.000137,0.000181,0.000109
ZBRA,0.000041,-0.000070,-0.000044,0.000166,0.000447,0.000182,0.000087,0.000336,0.000020,0.000179,...,1.828806e-04,0.000061,0.000292,0.000061,1.085247e-05,0.000078,0.002185,-0.000480,0.000014,0.000139
ZD,0.001105,0.000102,0.000177,-0.000047,-0.000009,0.000039,0.000213,0.000907,0.000269,0.000362,...,9.064468e-04,0.000273,-0.000078,0.000324,1.746693e-04,-0.000137,-0.000480,0.029861,0.000372,-0.000234
ZEUS,0.000429,-0.000096,0.000022,-0.000091,0.000060,0.000006,0.000013,-0.000200,0.000059,0.000042,...,-2.461123e-04,0.000083,0.000376,-0.000056,1.792474e-05,0.000181,0.000014,0.000372,0.002026,0.000069


In [132]:
cpu_dynamic_correlations[-1].index.name='ticker'
cpu_dynamic_correlations[-1].to_csv(os.path.join(os.getcwd(),'cpu_dynamic','cpu_dynamic_data_corr_'+str(year)+'.csv'))

In [133]:
dynamic_correlations[-1].index.name='ticker'
dynamic_correlations[-1].to_csv(os.path.join(os.getcwd(),'gpu_dynamic','gpu_dynamic_data_corr_'+str(year)+'.csv'))

In [134]:
pd.read_csv(os.path.join(os.getcwd(),'cpu_dynamic',f'cpu_dynamic_data_corr_{year}.csv'),index_col='ticker')

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
ticker,,,,,,,,,,,,,,,,,,,,,
A,0.003817,0.000140,0.000252,0.000154,0.000529,0.000242,0.000194,0.000526,-0.000017,0.000215,...,1.152240e-03,0.000034,0.000529,-0.000033,5.717272e-05,0.000283,0.000041,0.001105,0.000429,0.000189
AA,0.000140,0.001166,0.000055,-0.000056,-0.000125,0.000142,0.000182,-0.000118,0.000016,0.000027,...,4.190781e-04,0.000134,0.000180,-0.000014,8.759106e-05,0.000168,-0.000070,0.000102,-0.000096,0.000153
AAME,0.000252,0.000055,0.001877,-0.000044,0.000005,0.000063,0.000021,-0.000064,0.000046,0.000126,...,-2.868960e-05,-0.000088,0.000018,0.000104,3.912068e-05,-0.000010,-0.000044,0.000177,0.000022,0.000008
AAON,0.000154,-0.000056,-0.000044,0.001017,0.000039,0.000037,-0.000003,0.000125,0.000001,0.000010,...,1.080929e-04,0.000145,-0.000044,-0.000001,-2.206220e-05,-0.000076,0.000166,-0.000047,-0.000091,0.000002
AAPL,0.000529,-0.000125,0.000005,0.000039,0.002360,0.000122,-0.000099,0.000142,-0.000020,0.000084,...,6.767085e-04,0.000027,0.000527,0.000104,-4.272223e-05,0.000034,0.000473,-0.000009,0.000065,0.000129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.000283,0.000168,-0.000010,-0.000076,0.000034,0.000174,0.000038,0.000166,0.000016,0.000002,...,-2.254811e-07,0.000064,0.000307,0.000035,3.931895e-05,0.000777,0.000076,-0.000134,0.000179,0.000110
ZBRA,0.000041,-0.000070,-0.000044,0.000166,0.000473,0.000182,0.000097,0.000351,0.000020,0.000179,...,1.892892e-04,0.000061,0.000292,0.000061,1.084336e-05,0.000076,0.002185,-0.000480,0.000014,0.000139
ZD,0.001105,0.000102,0.000177,-0.000047,-0.000009,0.000039,0.000235,0.000922,0.000268,0.000362,...,1.011558e-03,0.000273,-0.000078,0.000324,1.745225e-04,-0.000134,-0.000480,0.029861,0.000372,-0.000234


In [135]:
pd.read_csv(os.path.join(os.getcwd(),'gpu_dynamic',f'gpu_dynamic_data_corr_{year}.csv'),index_col='ticker')

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
ticker,,,,,,,,,,,,,,,,,,,,,
A,0.003817,0.000140,0.000253,0.000154,0.000505,0.000242,0.000172,0.000505,-0.000017,0.000215,...,1.073485e-03,0.000034,0.000529,-0.000033,5.721862e-05,0.000281,0.000041,0.001105,0.000429,0.000189
AA,0.000140,0.001165,0.000055,-0.000056,-0.000115,0.000142,0.000150,-0.000110,0.000016,0.000027,...,4.155465e-04,0.000134,0.000180,-0.000014,8.760381e-05,0.000167,-0.000070,0.000102,-0.000096,0.000153
AAME,0.000253,0.000055,0.001877,-0.000044,0.000005,0.000063,0.000022,-0.000060,0.000046,0.000126,...,-3.561604e-05,-0.000088,0.000018,0.000104,3.915804e-05,-0.000010,-0.000044,0.000177,0.000022,0.000008
AAON,0.000154,-0.000056,-0.000044,0.001017,0.000034,0.000037,-0.000001,0.000117,0.000001,0.000010,...,9.811480e-05,0.000145,-0.000044,-0.000001,-2.208081e-05,-0.000079,0.000166,-0.000047,-0.000091,0.000002
AAPL,0.000505,-0.000115,0.000005,0.000034,0.002100,0.000114,-0.000071,0.000140,-0.000019,0.000081,...,6.173111e-04,0.000026,0.000496,0.000096,-4.053883e-05,0.000034,0.000447,-0.000009,0.000060,0.000122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.000281,0.000167,-0.000010,-0.000079,0.000034,0.000173,0.000032,0.000163,0.000016,0.000002,...,-3.604159e-07,0.000063,0.000312,0.000036,3.910462e-05,0.000787,0.000078,-0.000137,0.000181,0.000109
ZBRA,0.000041,-0.000070,-0.000044,0.000166,0.000447,0.000182,0.000087,0.000336,0.000020,0.000179,...,1.828806e-04,0.000061,0.000292,0.000061,1.085247e-05,0.000078,0.002185,-0.000480,0.000014,0.000139
ZD,0.001105,0.000102,0.000177,-0.000047,-0.000009,0.000039,0.000213,0.000907,0.000269,0.000362,...,9.064468e-04,0.000273,-0.000078,0.000324,1.746693e-04,-0.000137,-0.000480,0.029861,0.000372,-0.000234


In [136]:
dynamic_correlations[-1]

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
ticker,,,,,,,,,,,,,,,,,,,,,
A,0.003817,0.000140,0.000253,0.000154,0.000505,0.000242,0.000172,0.000505,-0.000017,0.000215,...,1.073485e-03,0.000034,0.000529,-0.000033,5.721862e-05,0.000281,0.000041,0.001105,0.000429,0.000189
AA,0.000140,0.001165,0.000055,-0.000056,-0.000115,0.000142,0.000150,-0.000110,0.000016,0.000027,...,4.155465e-04,0.000134,0.000180,-0.000014,8.760381e-05,0.000167,-0.000070,0.000102,-0.000096,0.000153
AAME,0.000253,0.000055,0.001877,-0.000044,0.000005,0.000063,0.000022,-0.000060,0.000046,0.000126,...,-3.561604e-05,-0.000088,0.000018,0.000104,3.915804e-05,-0.000010,-0.000044,0.000177,0.000022,0.000008
AAON,0.000154,-0.000056,-0.000044,0.001017,0.000034,0.000037,-0.000001,0.000117,0.000001,0.000010,...,9.811480e-05,0.000145,-0.000044,-0.000001,-2.208081e-05,-0.000079,0.000166,-0.000047,-0.000091,0.000002
AAPL,0.000505,-0.000115,0.000005,0.000034,0.002100,0.000114,-0.000071,0.000140,-0.000019,0.000081,...,6.173111e-04,0.000026,0.000496,0.000096,-4.053883e-05,0.000034,0.000447,-0.000009,0.000060,0.000122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.000281,0.000167,-0.000010,-0.000079,0.000034,0.000173,0.000032,0.000163,0.000016,0.000002,...,-3.604159e-07,0.000063,0.000312,0.000036,3.910462e-05,0.000787,0.000078,-0.000137,0.000181,0.000109
ZBRA,0.000041,-0.000070,-0.000044,0.000166,0.000447,0.000182,0.000087,0.000336,0.000020,0.000179,...,1.828806e-04,0.000061,0.000292,0.000061,1.085247e-05,0.000078,0.002185,-0.000480,0.000014,0.000139
ZD,0.001105,0.000102,0.000177,-0.000047,-0.000009,0.000039,0.000213,0.000907,0.000269,0.000362,...,9.064468e-04,0.000273,-0.000078,0.000324,1.746693e-04,-0.000137,-0.000480,0.029861,0.000372,-0.000234


In [137]:
len(cpu_returns)

2

In [138]:
#cpu_returns[-1]

In [139]:
cpu_returns[-1].index.name='ticker'
cpu_returns[-1].to_csv(os.path.join(os.getcwd(),'cpu_returns','cpu_returns_'+str(year)+'.csv'))

In [140]:
gpu_returns[-1].index.name='ticker'
gpu_returns[-1].to_csv(os.path.join(os.getcwd(),'gpu_returns','gpu_returns_'+str(year)+'.csv'))

In [141]:
pd.read_csv(os.path.join(os.getcwd(),'cpu_returns',f'cpu_returns_{year}.csv'),index_col='ticker')

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
ticker,,,,,,,,,,,,,,,,,,,,,
0,-0.076389,0.004633,0.027027,0.041667,-0.084310,-0.004201,-0.011628,-0.071429,0.000000,0.003077,...,-0.021053,0.000000,-0.046512,0.000000,-0.006885,-0.020100,-0.014428,-0.057692,0.000000,-0.048423
1,-0.062030,0.057648,0.026316,0.008889,0.014633,0.004219,0.000000,0.000000,0.000000,-0.012269,...,0.000000,0.017287,0.051491,-0.007353,0.005200,0.005127,0.019144,-0.040816,0.000000,-0.001183
2,-0.038076,-0.013081,-0.051282,0.004405,-0.086538,0.016807,0.000000,0.153846,0.000000,0.003106,...,0.010753,0.001307,-0.020618,0.000000,-0.006897,-0.008503,-0.054144,0.053191,-0.025974,0.014218
3,0.083333,-0.002946,0.027027,-0.035088,0.047369,0.033058,0.011765,0.000000,0.000000,0.003095,...,-0.042553,-0.002611,0.023684,0.000000,0.010417,-0.022298,-0.011098,-0.010101,0.066666,0.002337
4,0.060577,-0.002954,0.000000,0.000000,-0.017588,-0.018000,-0.023256,0.600000,0.000000,0.000000,...,0.044444,0.023560,-0.023136,0.000000,-0.006873,0.040351,0.033668,0.010204,0.025000,-0.018066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.064476,0.096456,0.066667,-0.006897,0.066667,0.002670,0.036232,0.051282,0.000000,0.044716,...,0.090226,0.039049,0.051948,-0.007195,-0.004238,-0.007767,0.025807,-0.142858,0.000000,0.060085
247,-0.021714,-0.007181,-0.093750,-0.020833,-0.020832,0.000000,0.006993,-0.048780,0.000000,-0.035019,...,0.048276,0.000170,-0.037037,-0.043478,-0.006383,-0.007828,-0.026730,-0.250001,0.014706,-0.004048
248,0.045561,-0.010850,0.000000,0.014185,0.008510,0.038615,0.041666,0.012821,0.074626,-0.002017,...,0.003289,0.060557,0.025641,0.046153,-0.008565,0.019724,0.033926,0.222223,0.000000,0.002032


In [142]:
pd.read_csv(os.path.join(os.getcwd(),'gpu_returns',f'gpu_returns_{year}.csv'),index_col='ticker')

,A,AA,AAME,AAON,AAPL,AB,ABCB,ABEO,ABEV,ABM,...,XOMA,XRAY,XRX,YORW,YPF,YUM,ZBRA,ZD,ZEUS,ZION
ticker,,,,,,,,,,,,,,,,,,,,,
0,-0.076389,0.004633,0.027027,0.041667,-0.084310,-0.004201,-0.011628,-0.071429,0.000000,0.003077,...,-0.021053,0.000000,-0.046512,0.000000,-0.006885,-0.020100,-0.014428,-0.057692,0.000000,-0.048423
1,-0.062030,0.057648,0.026316,0.008889,0.014633,0.004219,0.000000,0.000000,0.000000,-0.012269,...,0.000000,0.017287,0.051491,-0.007353,0.005200,0.005127,0.019144,-0.040816,0.000000,-0.001183
2,-0.038076,-0.013081,-0.051282,0.004405,-0.086538,0.016807,0.000000,0.153846,0.000000,0.003106,...,0.010753,0.001307,-0.020618,0.000000,-0.006897,-0.008503,-0.054144,0.053191,-0.025974,0.014218
3,0.083333,-0.002946,0.027027,-0.035088,0.047369,0.033058,0.011765,0.000000,0.000000,0.003095,...,-0.042553,-0.002611,0.023684,0.000000,0.010417,-0.022298,-0.011098,-0.010101,0.066666,0.002337
4,0.060577,-0.002954,0.000000,0.000000,-0.017588,-0.018000,-0.023256,0.600000,0.000000,0.000000,...,0.044444,0.023560,-0.023136,0.000000,-0.006873,0.040351,0.033668,0.010204,0.025000,-0.018066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.064476,0.096456,0.066667,-0.006897,0.066667,0.002670,0.036232,0.051282,0.000000,0.044716,...,0.090226,0.039049,0.051948,-0.007195,-0.004238,-0.007767,0.025807,-0.142858,0.000000,0.060085
247,-0.021714,-0.007181,-0.093750,-0.020833,-0.020832,0.000000,0.006993,-0.048780,0.000000,-0.035019,...,0.048276,0.000170,-0.037037,-0.043478,-0.006383,-0.007828,-0.026730,-0.250001,0.014706,-0.004048
248,0.045561,-0.010850,0.000000,0.014185,0.008510,0.038615,0.041666,0.012821,0.074626,-0.002017,...,0.003289,0.060557,0.025641,0.046153,-0.008565,0.019724,0.033926,0.222223,0.000000,0.002032
